<a href="https://colab.research.google.com/github/bbanzai88/SynBioCrow/blob/main/SynbioCrowV001.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This system is an automated end-to-end bio-retrosynthesis and design platform that integrates multiple computational tools into a unified LangGraph-based workflow. It identifies, evaluates, and simulates novel enzymatic pathways for the biosynthesis of target molecules, combining retrosynthetic search, enzyme selection, thermodynamic filtering, and metabolic simulation.

Key Capabilities:

*   Automated Retrosynthesis (RetroBioCat2)
*   Generates de novo biochemical pathways from a target molecule’s SMILES.
*   Performs Monte Carlo Tree Search (MCTS) over reaction rules to explore feasible routes.
*   Pathway Extraction & Annotation
*   Converts retrosynthesis outputs into structured reaction tables (substrates, products, scores).
*   Integrates similarity metrics, prior precedents, and enzyme reaction type metadata.
*   Thermodynamic & Ranking Modules
*   Uses dGPredictor / eQuilibrator to estimate Gibbs free energies (ΔG′).
*   Ranks pathways based on combined heuristic and thermodynamic feasibility scores.
*   Enzyme Identification (Selenzyme)
*   Predicts candidate enzymes for each step, optionally scraping Selenzyme for activity and organism data.
*   Metabolic Simulation (COBRApy / novoStoic)
*   Constructs toy stoichiometric models for top-ranked pathways.
*   Runs flux balance analysis (FBA) to estimate steady-state feasibility and pathway yield.
*   Sequence Ranking + Design of Experiments (DoE)
*   Build / Approval / Export Workflow
*   Produces pathway briefs, annotated CSVs/JSONs, simulation flux tables, and an export manifest for downstream integration or lab validation.
*   List item

Technical Architecture

*   LangGraph orchestration: modular “nodes” for each stage (retrosynthesis, extract, thermo, rank, simulate, selenzyme, seqrank, doe, build, approve, export).
*   Micromamba-based environment isolation ensures reproducible dependency management per run.
*   Streaming execution & checkpointed state allow resuming or extending pipelines dynamically.
*   Output artifacts (CSV, JSON, Markdown) capture every stage for transparency and reproducibility.

In [6]:
from typing import List, Dict, Optional, TypedDict

from typing_extensions import TypedDict, Annotated
from operator import add

class DBTLState(TypedDict, total=False):
    # constants (never “updated” again)
    run_id: Annotated[str, "const"]
    workdir: Annotated[str, "const"]
    target_smiles: Annotated[str, "const"]
    host: Annotated[str, "const"]

    # normal fields (LastValue by default)
    constraints: dict
    status: str
    last_node: str
    approved: bool
    signals: dict

    # append-only logs
    logs: Annotated[list[str], add]

    # your artifacts…
    retro_out_dir: str
    pathways_all_csv: str
    pathways_solved_csv: str
    pathways_all_json: str
    pathways_solved_json: str
    pathway_json_map: dict
    steps_plan_csv: str
    ranked_csv: str
    # etc…


In [4]:
# @title
!pip install langgraph langchain_core
from langgraph.graph import END, StateGraph
from langchain_core.runnables import RunnableLambda

def log(state, msg):
    return { "logs": (state.get("logs", []) + [msg]) }

import os, json, shutil, subprocess, sys, textwrap, time
from pathlib import Path
from typing import Dict

def _run(cmd, timeout=1800, env=None, cwd=None, check=True) -> subprocess.CompletedProcess:
    """Run a shell command with robust defaults and nice logging."""
    print(f"[run] {cmd}")
    res = subprocess.run(
        cmd,
        shell=True,
        cwd=cwd,
        env=env or os.environ.copy(),
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        timeout=timeout,
        check=check,
    )
    print(res.stdout)
    return res

def _ensure_dirs(*paths):
    for p in paths:
        Path(p).mkdir(parents=True, exist_ok=True)

def retrosynthesis_node(state: "DBTLState") -> "DBTLState":
    """
    End-to-end RetroBioCat-2 retrosynthesis node.
    Bootstraps micromamba, installs RetroBioCat2, runs MCTS,
    and saves pathway artifacts to workdir/retro_out.
    """
    import os, subprocess, textwrap, json, shutil, time, glob
    from pathlib import Path

    def _log(s, msg):
        s = {**s}
        s["logs"] = [*s.get("logs", []), msg]
        return s

    try:
        workdir = Path(state["workdir"]).expanduser().resolve()
        target = state.get("target_smiles")
        if not target:
            raise ValueError("retrosynthesis_node: state['target_smiles'] is required")

        cons = (state.get("constraints") or {})
        max_time = int(cons.get("max_search_time", 15))
        expanders = list(cons.get("expanders", ["retrobiocat"]))
        max_depth = cons.get("max_depth", None)
        force_rerun = bool(cons.get("rerun", False))

        run_dir = workdir
        retro_out = run_dir / "retro_out"
        retro_out.mkdir(parents=True, exist_ok=True)

        mm_root = run_dir / "micromamba"
        mm_bin = run_dir / "bin" / "micromamba"
        env_pref = mm_root / "envs" / "retrobiocat"

        state = _log(state, f"retrosynthesis: workdir={run_dir}")
        state = _log(state, f"retrosynthesis: target={target}")

        # ---- bootstrap micromamba ----
        if not mm_bin.exists():
            state = _log(state, "[rbc2] installing micromamba locally …")
            run_dir.mkdir(parents=True, exist_ok=True)
            subprocess.run(
                f"curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest -o {run_dir}/mm.tar.bz2",
                shell=True, check=True
            )
            subprocess.run(
                f"tar -xvjf {run_dir}/mm.tar.bz2 -C {run_dir} bin/micromamba",
                shell=True, check=True
            )

        env = os.environ.copy()
        env["MAMBA_ROOT_PREFIX"] = str(mm_root)

        # ---- create env if missing ----
        if force_rerun and env_pref.exists():
            shutil.rmtree(env_pref, ignore_errors=True)
        if not env_pref.exists():
            state = _log(state, "[rbc2] creating env …")
            subprocess.run(
                f"{mm_bin} create -y -p {env_pref} -c conda-forge python=3.10 pip",
                shell=True, env=env, check=True
            )

        # ---- install RBC2 + pins ----
        state = _log(state, "[rbc2] installing RetroBioCat-2 + pins …")
        cmds = [
            f"{mm_bin} run -p {env_pref} python -m pip install --upgrade pip setuptools wheel",
            f"{mm_bin} run -p {env_pref} python -m pip install https://github.com/willfinnigan/RetroBioCat-2/archive/refs/heads/main.zip "
            "|| "
            f"{mm_bin} run -p {env_pref} python -m pip install 'git+https://github.com/willfinnigan/RetroBioCat-2.git'",
            f"{mm_bin} run -p {env_pref} python -m pip install 'pydantic<2' 'networkx<3' tqdm requests",
        ]
        for c in cmds:
            subprocess.run(c, shell=True, env=env, check=True)

        # ---- inline Python for MCTS ----
        py = textwrap.dedent(f"""
            import os, json, csv
            from pathlib import Path
            from rbc2 import MCTS, get_expanders

            retro_out = Path(r"{retro_out}")
            retro_out.mkdir(parents=True, exist_ok=True)

            target_smi = r"{target}"
            expanders  = get_expanders({json.dumps(expanders)})

            mcts = MCTS(target_smi, expanders)
            mcts.config.max_search_time = int({max_time})
            {"mcts.config.max_depth = int(%d)" % int(max_depth) if max_depth is not None else ""}

            print("[rbc2] Running MCTS …")
            mcts.run()

            all_paths    = mcts.get_all_pathways()
            solved_paths = mcts.get_solved_pathways()

            def pathway_rows(pwy, tag):
                rows = []
                for i, rxn in enumerate(pwy.reactions, start=1):
                    rows.append({{
                        "pathway_tag": tag,
                        "step_idx": i,
                        "reaction_smiles": rxn.reaction_smiles(),
                        "product": rxn.product,
                        "substrates": " . ".join(rxn.substrates),
                        "rxn_type": rxn.rxn_type,
                        "name": rxn.name,
                        "score": rxn.score,
                    }})
                return rows

            def write_bundle(paths, prefix):
                import pandas as pd
                all_rows = []
                tag_to_file = {{}}
                for k, pwy in enumerate(paths, start=1):
                    tag = f"P{{k:03d}}"
                    pjson = retro_out / f"{{tag}}.json"
                    with open(pjson, "w", encoding="utf-8") as f:
                        json.dump(pwy.save(), f, indent=2)
                    tag_to_file[tag] = str(pjson)
                    all_rows.extend(pathway_rows(pwy, tag))
                if all_rows:
                    df = pd.DataFrame(all_rows)
                    df.to_csv(retro_out / f"pathways_{{prefix}}_steps.csv", index=False)
                    with open(retro_out / f"pathways_{{prefix}}_steps.json", "w", encoding="utf-8") as f:
                        json.dump(all_rows, f, indent=2)
                return tag_to_file

            tag_map_all = write_bundle(all_paths, "all")
            tag_map_sol = write_bundle(solved_paths, "solved")

            print(f"[rbc2] Total pathways: {{len(all_paths)}} | Solved: {{len(solved_paths)}}")
            print("[rbc2] Outputs written to:", retro_out)
        """).strip()

        t0 = time.time()
        subprocess.run(
            f"""{mm_bin} run -p {env_pref} python - <<'PY'\n{py}\nPY""",
            shell=True, env=env, check=True
        )
        dt = time.time() - t0
        state = _log(state, f"[rbc2] finished in {dt:.1f}s")

        # ---- collect outputs ----
        pathways_all_csv = str(retro_out / "pathways_all_steps.csv")
        pathways_solved_csv = str(retro_out / "pathways_solved_steps.csv")
        pathways_all_json = str(retro_out / "pathways_all_steps.json")
        pathways_solved_json = str(retro_out / "pathways_solved_steps.json")

        pjson_files = glob.glob(str(retro_out / "P*.json"))
        tag_map = {Path(p).stem: str(Path(p)) for p in pjson_files}

        new_state = {
            **state,
            "retro_out_dir": str(retro_out),
            "pathways_all_csv": pathways_all_csv if os.path.exists(pathways_all_csv) else None,
            "pathways_solved_csv": pathways_solved_csv if os.path.exists(pathways_solved_csv) else None,
            "pathways_all_json": pathways_all_json if os.path.exists(pathways_all_json) else None,
            "pathways_solved_json": pathways_solved_json if os.path.exists(pathways_solved_json) else None,
            "pathway_json_map": tag_map,
        }
        new_state["logs"] = [*new_state.get("logs", []), "retrosynthesis done"]
        return new_state

    except subprocess.CalledProcessError as e:
        err = f"retrosynthesis_node: subprocess failed (code={e.returncode})"
        new_state = {**state, "error": err}
        new_state["logs"] = [*new_state.get("logs", []), err, str(e)]
        return new_state

    except Exception as e:
        err = f"retrosynthesis_node: {type(e).__name__}: {e}"
        new_state = {**state, "error": err}
        new_state["logs"] = [*new_state.get("logs", []), err]
        return new_state



def extract_pathways_node(state: "DBTLState") -> "DBTLState":
    """
    Build enzyme-planning artifacts from RetroBioCat-2 outputs:
      - steps_enzyme_plan.csv (per-step details incl. enzyme classes, precedents, Selenzyme links)
      - pathways_ranked_no_thermo.csv (RBC2 + precedent-based ranking; no thermodynamics)
    Reads:
      - {workdir}/retro_out/pathways_solved_steps.csv (or .../pathways_all_steps.csv)
      - {workdir}/retro_out/Pxxx.json (per-pathway, if present)
    """
    workdir = Path(state["workdir"])
    retro_out = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # ---- pick CSV (solved preferred, else all) ----
    csv_solved = retro_out / "pathways_solved_steps.csv"
    csv_all    = retro_out / "pathways_all_steps.csv"
    csv_in = csv_solved if csv_solved.exists() else csv_all
    if not csv_in.exists():
        raise FileNotFoundError(f"No pathways CSV found under {retro_out}")

    steps_df = pd.read_csv(csv_in)

    # ---- pathway -> json map (prefer state, else scan) ----
    json_map: Dict[str,str] = dict(state.get("pathway_json_map") or {})
    if not json_map:
        for p in glob.glob(str(retro_out / "P*.json")):
            tag = Path(p).stem  # P001
            json_map[tag] = p

    host_org = state.get("host") or "Escherichia coli"

    def selenzyme_url_from_rxn(rxn_smiles: str, org: str) -> str:
        return (
            "https://selenzyme.synbiochem.co.uk/selenzy/selenzy?reaction_smiles="
            + quote_plus(rxn_smiles)
            + "&organism="
            + quote_plus(org)
        )

    # ---- expand per-step details from Pxxx.json ----
    records = []
    for _, r in steps_df.iterrows():
        tag   = str(r.get("pathway_tag"))
        idx   = int(r.get("step_idx"))
        rxn   = str(r.get("reaction_smiles",""))
        prod  = str(r.get("products", r.get("product","")))
        subs  = str(r.get("substrates",""))

        rxn_type = r.get("rxn_type","")
        name     = r.get("name","")
        rbc2_score = r.get("score", float("nan"))

        possible_enzymes = []
        enzyme_choices = []
        selected_enzyme = ""
        precedents = []

        pj = json_map.get(tag)
        if pj and os.path.exists(pj):
            try:
                data = json.load(open(pj, "r"))
                if isinstance(data, list) and 1 <= idx <= len(data):
                    d = data[idx - 1]
                    rxn_type = d.get("rxn_type", rxn_type)
                    name     = d.get("name", name)
                    tm = d.get("template_metadata", {}) or {}
                    if tm:
                        tkey = next(iter(tm.keys()))
                        tmeta = tm.get(tkey, {}) or {}
                        possible_enzymes = (tmeta.get("possible_enzymes") or [])
                        choices = (tmeta.get("enzyme_choices") or [])
                        enzyme_choices = [
                            " / ".join(x) if isinstance(x, (list, tuple)) else str(x)
                            for x in choices
                        ]
                        selected_enzyme = tmeta.get("selected_enzyme") or ""
                    precedents = d.get("precedents") or []
            except Exception:
                pass  # keep whatever we already have

        # top-3 precedent summary
        prec_sorted = sorted(precedents, key=lambda x: x.get("similarity", 0), reverse=True)[:3]
        prec_summ, best_sim = [], float("nan")
        for p in prec_sorted:
            sim = p.get("similarity", float("nan"))
            dat = p.get("data", {}) or {}
            doi = dat.get("html_doi", "") or dat.get("doi", "")
            enz = dat.get("enzyme_name", "") or p.get("name", "")
            cite = dat.get("short_citation", "")
            try:
                prec_summ.append(f"{enz} | sim={float(sim):.3f} | {cite} | {doi}")
            except Exception:
                prec_summ.append(f"{enz} | sim={sim} | {cite} | {doi}")
        if prec_sorted:
            best_sim = prec_sorted[0].get("similarity", float("nan"))

        records.append({
            "pathway_tag": tag,
            "step_idx": idx,
            "rxn_type": rxn_type,
            "retrobiocat_reaction": name,
            "reaction_smiles": rxn,
            "substrates": subs,
            "products": prod,
            "rbc2_score": rbc2_score,
            "selected_enzyme": selected_enzyme,
            "possible_enzymes": " ; ".join(possible_enzymes),
            "enzyme_choices": " ; ".join(enzyme_choices),
            "precedent_top3": " || ".join(prec_summ),
            "precedent_best_similarity": best_sim,
            "selenzyme_url": selenzyme_url_from_rxn(rxn, host_org),
        })

    plan = pd.DataFrame(records).sort_values(["pathway_tag","step_idx"]).reset_index(drop=True)

    # ---- pathway ranking (no thermodynamics) ----
    tmp = plan.copy()
    tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
    tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)

    agg = tmp.groupby("pathway_tag").agg(
        steps_count=("step_idx","max"),
        sum_rbc2=("rbc2_score","sum"),
        sum_prec_sim=("precedent_best_similarity","sum"),
    ).reset_index()

    # heuristic: favor higher RBC2 + stronger precedents + fewer steps
    agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
    ranked = agg.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False, False, False, True]
    ).reset_index(drop=True)

    # ---- write outputs ----
    steps_plan_csv = str(finish_out / "steps_enzyme_plan.csv")
    ranked_csv     = str(finish_out / "pathways_ranked_no_thermo.csv")
    plan.to_csv(steps_plan_csv, index=False)
    ranked.to_csv(ranked_csv, index=False)

    # ---- return updated state ----
    new_state = {
        **state,
        "steps_plan_csv": steps_plan_csv,
        "ranked_csv": ranked_csv,
        "thermo_method": "none",
    }
    new_state["logs"] = state.get("logs", []) + [
        f"extracted pathways ({len(plan)} step rows, {len(ranked)} pathways)",
        f"steps_plan={steps_plan_csv}",
        f"ranked_no_thermo={ranked_csv}",
    ]
    return new_state
import os, subprocess, json
from pathlib import Path
import textwrap

def thermo_score_node(state: "DBTLState") -> "DBTLState":
    """
    Lightweight 'match-only' thermo pass:
      - No ChemAxon, no ΔG′° numbers (columns are NaN/False).
      - Uses the SAME micromamba env created by retrosynthesis_node.
      - Emits:
          retro_finish_out/steps_annotated.csv
          retro_finish_out/pathways_ranked.csv
      - Sets thermo_method='matchonly' and logs coverage like your working run.
    """
    workdir    = Path(state["workdir"])
    mm_root    = workdir / "micromamba"
    env_prefix = mm_root / "envs" / "retrobiocat"  # same env as retrosynthesis_node
    retro_out  = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # choose input table (prefer extractor output)
    steps_plan_csv = state.get("steps_plan_csv")
    if not steps_plan_csv or not Path(steps_plan_csv).exists():
        # fall back to solved/all from retro_out
        csv_solved = retro_out / "pathways_solved_steps.csv"
        csv_all    = retro_out / "pathways_all_steps.csv"
        steps_plan_csv = str(csv_solved if csv_solved.exists() else csv_all)

    # outputs
    steps_out = str(finish_out / "steps_annotated.csv")
    rank_out  = str(finish_out / "pathways_ranked.csv")

    # python payload (runs INSIDE the retrobiocat env)
    py = f"""\
import os, math, pandas as pd
from pathlib import Path

IN  = Path(r"{steps_plan_csv}")
OUT_STEPS = Path(r"{steps_out}")
OUT_RANK  = Path(r"{rank_out}")

if not IN.exists():
    raise FileNotFoundError(f"Input steps table not found: {{IN}}")

df = pd.read_csv(IN)

# Ensure required columns exist (from extractor), or create dummies
for col in ["pathway_tag","step_idx","reaction_smiles","substrates","products",
            "rbc2_score","precedent_best_similarity","selenzyme_url"]:
    if col not in df.columns:
        df[col] = "" if col not in ["rbc2_score","precedent_best_similarity"] else 0.0

# --- Parse reaction_smiles to count participants (match stats only) ---
def parse_rxn(r):
    if not isinstance(r, str) or ">>" not in r: return [], []
    L,R = r.split(">>",1)
    subs=[s for s in L.split(".") if s]; prods=[p for p in R.split(".") if p]
    return subs, prods

participants = 0
matched = 0   # 'match-only' mode → we don't do DB lookup; keep 0 like your working run
subs_counts, prod_counts = [], []
for rxn in df["reaction_smiles"].astype(str):
    subs, prods = parse_rxn(rxn)
    subs_counts.append(len(subs))
    prod_counts.append(len(prods))
    participants += len(subs) + len(prods)

df["subs_count"] = subs_counts
df["prods_count"] = prod_counts

# --- Thermo placeholders (no ChemAxon / no ΔG numbers) ---
df["dGprime_kJ_per_mol"] = float("nan")
df["uncert_kJ_per_mol"]  = float("nan")
df["thermo_pass"]        = False
df["equilibrator_formula"] = ""  # keep blank; earlier 'coco:...' strings were info-only

# Save annotated steps
df.to_csv(OUT_STEPS, index=False)

# --- Ranking (identical heuristic to extractor) ---
tmp = df.copy()
tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)

agg = tmp.groupby("pathway_tag").agg(
    steps_count=("step_idx","max"),
    sum_rbc2=("rbc2_score","sum"),
    sum_prec_sim=("precedent_best_similarity","sum"),
).reset_index()

agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
ranked = agg.sort_values(["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
                         ascending=[False,False,False,True]).reset_index(drop=True)
ranked.to_csv(OUT_RANK, index=False)

print("Match stats:", {{"participants": participants, "matched": matched,
                       "steps_full_match": 0, "total_steps": int(df.shape[0])}})
print("Saved:")
print(" ", OUT_STEPS)
print(" ", OUT_RANK)
"""

    # run inside the existing env
    micromamba = workdir / "bin" / "micromamba"
    if not micromamba.exists():
        # In case node is called standalone, try system micromamba
        micromamba = "micromamba"

    env = os.environ.copy()
    env["MAMBA_ROOT_PREFIX"] = str(mm_root)

    # pandas is already in the env from earlier steps, but be forgiving:
    try:
        subprocess.run(
            f"""{micromamba} run -p "{env_prefix}" python - <<'PY'\n{py}\nPY""",
            shell=True, check=True, text=True, env=env
        )
    except subprocess.CalledProcessError as e:
        # As a fallback, try running with system Python (Colab) if env missing.
        subprocess.run(f"python - <<'PY'\n{py}\nPY", shell=True, check=True, text=True)

    # compute coverage flag (we reported matched=0 in-script)
    coverage = 0
    method = "matchonly"

    # update state
    new_state = {
        **state,
        "thermo_method": method,
        "steps_annotated_csv": steps_out,
        "ranked_csv": rank_out,   # keep same key as extractor (downstream nodes can reuse)
    }
    new_state["logs"] = state.get("logs", []) + [
        f"thermo: {method}, coverage={coverage}",
        f"steps_annotated={steps_out}",
        f"ranked={rank_out}",
    ]
    return new_state

import pandas as pd
from pathlib import Path
import os

def rank_node(state: "DBTLState") -> "DBTLState":
    """
    Merge thermo info (if available) into pathway ranking.
    Ranking logic:
      - If thermo columns present and any thermo_pass=True:
          prefer pathways with all_steps_pass=True, lower sum_dGprime
      - Else fall back to rbc2_score + precedent_best_similarity - step penalty
    Outputs:
      {workdir}/retro_finish_out/pathways_ranked_final.csv
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # Input sources
    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not steps_csv or not Path(steps_csv).exists():
        raise FileNotFoundError("rank_node: steps CSV not found in state")

    df = pd.read_csv(steps_csv)
    df.columns = [c.strip() for c in df.columns]

    # --- detect thermo presence ---
    has_thermo = any(c in df.columns for c in ["dGprime_kJ_per_mol", "thermo_pass"])
    thermo_mode = bool(has_thermo and df["thermo_pass"].any())

    # --- normalize numeric fields ---
    df["rbc2_score"] = pd.to_numeric(df.get("rbc2_score", 0.0), errors="coerce").fillna(0.0)
    df["precedent_best_similarity"] = pd.to_numeric(df.get("precedent_best_similarity", 0.0),
                                                    errors="coerce").fillna(0.0)
    if "dGprime_kJ_per_mol" in df:
        df["dGprime_kJ_per_mol"] = pd.to_numeric(df["dGprime_kJ_per_mol"], errors="coerce")

    # --- aggregate by pathway ---
    group = df.groupby("pathway_tag", dropna=True)

    if thermo_mode:
        # Include thermo-based weighting
        ranked = (
            group.agg(
                steps_count=("step_idx","max"),
                sum_rbc2=("rbc2_score","sum"),
                sum_prec_sim=("precedent_best_similarity","sum"),
                thermo_pass_steps=("thermo_pass","sum"),
                sum_dGprime=("dGprime_kJ_per_mol","sum"),
            )
            .reset_index()
        )
        ranked["all_steps_pass"] = ranked["thermo_pass_steps"] == ranked["steps_count"]
        ranked["rank_score"] = (
            ranked["sum_rbc2"]
            + 0.2 * ranked["sum_prec_sim"]
            - 0.1 * ranked["steps_count"]
            - 0.001 * ranked["sum_dGprime"].fillna(0.0)
            + ranked["all_steps_pass"].astype(float) * 0.5
        )
        method = "rbc2+precedent+thermo"
    else:
        # Fallback: RBC2 + precedents + shortness
        ranked = (
            group.agg(
                steps_count=("step_idx","max"),
                sum_rbc2=("rbc2_score","sum"),
                sum_prec_sim=("precedent_best_similarity","sum"),
            )
            .reset_index()
        )
        ranked["rank_score"] = (
            ranked["sum_rbc2"] + 0.2 * ranked["sum_prec_sim"] - 0.1 * ranked["steps_count"]
        )
        method = "rbc2+precedent"

    ranked = ranked.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False, False, False, True],
    ).reset_index(drop=True)

    # --- save output ---
    out_csv = str(finish_out / "pathways_ranked_final.csv")
    ranked.to_csv(out_csv, index=False)

    new_state = {
        **state,
        "ranked_final_csv": out_csv,
        "ranking_method": method,
    }
    new_state["logs"] = state.get("logs", []) + [
        f"ranked pathways ({method}, {len(ranked)} entries)",
        f"ranked_final={out_csv}",
    ]
    return new_state

import os, time, re, json
from pathlib import Path
from typing import Optional, Dict, List, Tuple
import pandas as pd

def selenzyme_node(state: "DBTLState") -> "DBTLState":
    """
    Collect Selenzyme links from the step table and (optionally) scrape top sequence
    suggestions from each page. Robust to offline/blocked environments — will fall back
    to counting links only.

    Inputs (preferred to fallback):
      - steps_annotated_csv (from thermo node) OR steps_plan_csv (from extractor)

    Outputs:
      - {workdir}/retro_finish_out/selenzyme_scrape.csv   (only if rows were scraped)
      - state["selenzyme_rows"] = number of scraped sequence rows (0 if none)
      - state["scraped_sequences_csv"] = path or None
      - log entries
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # ---- choose input table ----
    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not steps_csv or not Path(steps_csv).exists():
        raise FileNotFoundError("selenzyme_node: no steps CSV found in state")

    steps = pd.read_csv(steps_csv)
    if "selenzyme_url" not in steps.columns:
        # nothing to do — return graceful "0 rows"
        new_state = {
            **state,
            "selenzyme_rows": 0,
            "scraped_sequences_csv": None,
        }
        new_state["logs"] = state.get("logs", []) + [
            "selenzyme: no 'selenzyme_url' column; skipped"
        ]
        return new_state

    # ---- dedupe URLs, keep mapping back to step for provenance ----
    url_map: Dict[str, List[Tuple[str,int]]] = {}
    for _, r in steps.iterrows():
        url = str(r.get("selenzyme_url") or "").strip()
        if not url:
            continue
        tag = str(r.get("pathway_tag"))
        idx = int(r.get("step_idx", 0))
        url_map.setdefault(url, []).append((tag, idx))

    unique_urls = list(url_map.keys())

    # ---- scraping controls (polite defaults) ----
    cfg = (state.get("constraints", {}) or {}).get("selenzyme", {}) or {}
    do_scrape: bool   = bool(cfg.get("scrape", True))        # default True
    max_urls: int     = int(cfg.get("max_urls", 40))         # cap the number of hits
    timeout_s: int    = int(cfg.get("timeout_s", 15))
    sleep_s: float    = float(cfg.get("sleep_s", 1.0))

    # limit how many we actually hit (politeness / speed)
    urls_to_hit = unique_urls[:max_urls]

    scraped_rows: List[Dict] = []
    blocked_reason: Optional[str] = None

    def _try_scrape(url: str) -> List[Dict]:
        """
        Very light HTML scraper for Selenzyme tables. We keep this tolerant:
        - Works even if structure changes (regex fallbacks).
        - Returns empty list if blocked / unexpected layout.
        """
        try:
            # Lazy import so node works without requests/bs4 installed
            import requests
            from bs4 import BeautifulSoup  # type: ignore

            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
                              "AppleWebKit/537.36 (KHTML, like Gecko) "
                              "Chrome/122.0 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "Connection": "keep-alive",
            }
            resp = requests.get(url, headers=headers, timeout=timeout_s)
            if resp.status_code in (403, 429):
                raise RuntimeError(f"HTTP {resp.status_code}")
            if not resp.ok or not resp.text:
                return []
            html = resp.text
            soup = BeautifulSoup(html, "html.parser")

            # Look for a table with sequence hits. Selenzyme often renders a main hits table.
            # We search broadly: any table with headers that mention UniProt/Organism/Score/EC/Name.
            table = None
            for t in soup.find_all("table"):
                ths = [th.get_text(strip=True).lower() for th in t.find_all("th")]
                joined = " ".join(ths)
                if any(k in joined for k in ["uniprot", "organism", "score", "ec", "enzyme"]):
                    table = t
                    break

            rows_local: List[Dict] = []
            if table:
                # Extract header positions
                headers = [th.get_text(strip=True) for th in table.find_all("th")]
                # Build index map (case-insensitive)
                idx_map = {h.lower(): i for i, h in enumerate(headers)}

                def _get(td_list, key_variants):
                    for k in key_variants:
                        i = idx_map.get(k)
                        if i is not None and i < len(td_list):
                            return td_list[i].get_text(strip=True)
                    return ""

                for tr in table.find_all("tr"):
                    tds = tr.find_all("td")
                    if not tds:
                        continue
                    # Try a few common header variants
                    acc = _get(tds, ["uniprot", "accession", "uniprot id"])
                    org = _get(tds, ["organism", "source", "species"])
                    scr = _get(tds, ["score", "similarity", "sim"])
                    ec  = _get(tds, ["ec", "ec number"])
                    enz = _get(tds, ["enzyme", "name", "protein"])

                    # Clean accession (extract from anchor if present)
                    if not acc:
                        a = tds[0].find("a")
                        if a and a.get_text(strip=True):
                            acc = a.get_text(strip=True)
                    # Try a regex if still empty
                    if not acc:
                        m = re.search(r"[A-NR-Z0-9]{6,10}", tr.get_text(" ", strip=True))
                        if m:
                            acc = m.group(0)

                    # Normalize score to float if possible
                    try:
                        scr_val = float(re.sub("[^0-9.+-eE]", "", scr)) if scr else None
                    except Exception:
                        scr_val = None

                    rows_local.append({
                        "accession": acc,
                        "organism": org,
                        "score": scr_val if scr_val is not None else scr,
                        "ec": ec,
                        "enzyme_name": enz,
                    })

            # If table not found, do a broad regex fallback to pick UniProt-like codes.
            if not rows_local:
                # extract accession-like tokens to avoid returning nothing
                tokens = re.findall(r"\b[OPQ][0-9][A-Z0-9]{3}[0-9]\b|\b[A-NR-Z0-9]{6}\b", html)
                rows_local = [{"accession": t, "organism": "", "score": "", "ec": "", "enzyme_name": ""}
                              for t in dict.fromkeys(tokens)]  # dedupe preserve order

            return rows_local

        except Exception as e:
            nonlocal blocked_reason
            blocked_reason = str(e)
            return []

    if do_scrape and len(urls_to_hit) > 0:
        for i, url in enumerate(urls_to_hit, 1):
            rows_local = _try_scrape(url)
            # Attach provenance (all steps that used this URL)
            for (tag, idx) in url_map.get(url, []):
                for r in rows_local:
                    scraped_rows.append({
                        "pathway_tag": tag,
                        "step_idx": idx,
                        "selenzyme_url": url,
                        **r
                    })
            # polite delay
            if i < len(urls_to_hit):
                try:
                    time.sleep(sleep_s)
                except Exception:
                    pass

    # ---- Write output if we have any rows ----
    scraped_csv_path: Optional[str] = None
    if scraped_rows:
        out_csv = finish_out / "selenzyme_scrape.csv"
        pd.DataFrame(scraped_rows, columns=[
            "pathway_tag","step_idx","selenzyme_url",
            "accession","organism","score","ec","enzyme_name"
        ]).to_csv(out_csv, index=False)
        scraped_csv_path = str(out_csv)

    # ---- Compose state & logs ----
    total_links = len(unique_urls)
    rows = len(scraped_rows)

    log_lines = [
        f"selenzyme: links={total_links}, scraped_rows={rows}, "
        f"mode={'scrape' if do_scrape else 'count-only'}"
    ]
    if blocked_reason and rows == 0 and do_scrape:
        log_lines.append(f"selenzyme: scrape blocked or failed ({blocked_reason}); fell back to count-only")

    new_state = {
        **state,
        "selenzyme_rows": rows,
        "scraped_sequences_csv": scraped_csv_path,
    }
    new_state["logs"] = state.get("logs", []) + log_lines
    return new_state

import os, re, math
from pathlib import Path
from typing import Dict, List, Optional
import pandas as pd

def sequence_rank_node(state: "DBTLState") -> "DBTLState":
    """
    Rank Selenzyme-scraped sequences per step:
      - Deduplicate by UniProt accession (keep best-scoring row)
      - Prioritize bacterial organisms and preferred hosts
      - Keep top-N per step (configurable)
      - Emit a global shortlist (union of all per-step top-N; de-duped)

    Inputs (from state):
      - scraped_sequences_csv : path from selenzyme_node (required)
      - steps_plan_csv        : optional; used only for sorting/validation

    Constraints (optional):
      state["constraints"]["sequence_rank"] = {
         "top_n_per_step": 5,
         "preferred_hosts": ["Escherichia coli", "Bacillus subtilis"],
         "blacklist_regex": "(mitochondrial|chloroplast)",
         "extra_bacterial_genera": ["Pseudomonas","Corynebacterium","Rhodococcus"]
      }
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    scraped_csv = state.get("scraped_sequences_csv")
    if not scraped_csv or not Path(scraped_csv).exists():
        # graceful no-op
        new_state = { **state }
        new_state["logs"] = state.get("logs", []) + [
            "sequence_rank: no scraped_sequences_csv; skipped"
        ]
        return new_state

    steps_csv = state.get("steps_plan_csv")
    steps_order = None
    if steps_csv and Path(steps_csv).exists():
        try:
            _steps = pd.read_csv(steps_csv)
            steps_order = (
                _steps[["pathway_tag","step_idx"]]
                .drop_duplicates()
                .sort_values(["pathway_tag","step_idx"])
            )
        except Exception:
            steps_order = None

    cfg: Dict = (state.get("constraints", {}) or {}).get("sequence_rank", {}) or {}
    TOP_N = int(cfg.get("top_n_per_step", 5))
    preferred_hosts = set(cfg.get("preferred_hosts", []))
    extra_bact = set(cfg.get("extra_bacterial_genera", ["Pseudomonas","Corynebacterium","Rhodococcus","Acinetobacter"]))
    blacklist_re = re.compile(cfg.get("blacklist_regex", r"(mitochondrial|chloroplast)"), flags=re.I)

    # --- Load scraped rows ---
    df = pd.read_csv(scraped_csv)
    # Normalize expected columns
    for col in ["pathway_tag","step_idx","selenzyme_url","accession","organism","score","ec","enzyme_name"]:
        if col not in df.columns:
            df[col] = "" if col not in ["score","step_idx"] else (0.0 if col=="score" else 0)

    # Make sure types are sane
    df["step_idx"] = pd.to_numeric(df["step_idx"], errors="coerce").fillna(0).astype(int)
    # score may be non-numeric (strings) → parse to float when possible
    def _to_float(x):
      try:
        if pd.isna(x): return math.nan
        return float(str(x).strip())
      except Exception:
        return math.nan
    df["score_num"] = df["score"].apply(_to_float)

    # --- Heuristics ---
    # 1) Bacterial vs non-bacterial guess
    # crude but effective: mark as bacteria if organism matches common bacterial genera
    bacterial_keywords = [
        "bacter", "bacillus","escherichia","pseudomonas","corynebacterium","rhodococcus",
        "acinetobacter","enterobacter","salmonella","shigella","staphylococcus","streptococcus",
        "lactobacillus","lactococcus","klebsiella","burkholderia","clostridium","vibrio",
        "shewanella","xanthomonas","mycobacterium","yersinia","pasteurella","azotobacter",
        "cupriavidus","caulobacter","alcaligenes","sphingomonas","terranovum"
    ] + [g.lower() for g in extra_bact]
    euk_exclude = [
        "homo","mus","rattus","danio","drosophila","bos","gallus","arabidopsis","oryza","zea",
        "nicotiana","saccharomyces","pichia","kluyveromyces","aspergillus","neurospora","candida"
    ]

    def is_bacterial(org: str) -> bool:
        org_l = (org or "").lower()
        if any(k in org_l for k in euk_exclude):
            return False
        if any(k in org_l for k in bacterial_keywords):
            return True
        # default unknowns to False (conservative)
        return False

    df["is_bacterial"] = df["organism"].apply(is_bacterial)

    # 2) Preferred host bonus (exact contains)
    def in_preferred(org: str) -> bool:
        if not preferred_hosts: return False
        for ph in preferred_hosts:
            if ph.lower() in (org or "").lower():
                return True
        return False
    df["is_preferred_host"] = df["organism"].apply(in_preferred)

    # 3) Blacklist penalty
    df["is_blacklisted"] = df["organism"].apply(lambda x: bool(blacklist_re.search(str(x))) if str(x) else False)

    # 4) Accession quality hint (Swiss-Prot often 6 chars; but we won't assume)
    def acc_quality(acc: str) -> int:
        s = (acc or "").strip()
        # basic sanity: 6-10 alphanum looks like a proper code
        return 1 if re.fullmatch(r"[A-NR-Z0-9]{6,10}", s) else 0
    df["acc_quality"] = df["accession"].apply(acc_quality)

    # --- Composite score ---
    # Start with normalized numeric score in [0,1]; if NaN, default 0.5
    s = df["score_num"]
    # normalize by per-step distribution when present, else by global
    if s.notna().sum() > 0:
        # per-step min-max normalization (robust when distributions differ)
        def per_step_norm(g):
            vals = g["score_num"]
            vmin = vals.min(skipna=True)
            vmax = vals.max(skipna=True)
            if pd.isna(vmin) or pd.isna(vmax) or vmin == vmax:
                return pd.Series([0.5]*len(g), index=g.index)
            return (vals - vmin) / (vmax - vmin)
        df["score_norm"] = df.groupby(["pathway_tag","step_idx"], dropna=False, as_index=False).apply(per_step_norm).reset_index(level=[0,1], drop=True)
        df["score_norm"] = df["score_norm"].fillna(0.5)
    else:
        df["score_norm"] = 0.5

    # bonuses/penalties
    df["bonus_bacterial"] = df["is_bacterial"].astype(float) * 0.20
    df["bonus_preferred"] = df["is_preferred_host"].astype(float) * 0.10
    df["bonus_acc"]       = df["acc_quality"].astype(float) * 0.05
    df["pen_blacklist"]   = df["is_blacklisted"].astype(float) * 0.30  # big penalty

    df["rank_score"] = df["score_norm"] + df["bonus_bacterial"] + df["bonus_preferred"] + df["bonus_acc"] - df["pen_blacklist"]

    # --- Deduplicate by accession (keep best row overall) ---
    df = df.sort_values(["accession","rank_score"], ascending=[True, False])
    df_dedup = df.drop_duplicates(subset=["accession"], keep="first")

    # --- Rank within each step & keep top-N ---
    top_by_step = (
        df_dedup
        .sort_values(["pathway_tag","step_idx","rank_score"], ascending=[True, True, False])
        .groupby(["pathway_tag","step_idx"], dropna=False)
        .head(TOP_N)
        .reset_index(drop=True)
    )

    # Add UniProt URL
    def uniprot_url(acc: str) -> str:
        acc = str(acc).strip()
        return f"https://www.uniprot.org/uniprotkb/{acc}" if acc else ""
    top_by_step["uniprot_url"] = top_by_step["accession"].apply(uniprot_url)

    # Reorder columns nicely
    cols = [
        "pathway_tag","step_idx","accession","organism","ec","enzyme_name",
        "score","score_num","score_norm",
        "is_bacterial","is_preferred_host","is_blacklisted","acc_quality",
        "bonus_bacterial","bonus_preferred","bonus_acc","pen_blacklist",
        "rank_score","selenzyme_url","uniprot_url"
    ]
    cols = [c for c in cols if c in top_by_step.columns]  # guard
    top_by_step = top_by_step[cols]

    # Keep a global shortlist (unique accessions across all steps, highest rank first)
    shortlist = (
        top_by_step
        .sort_values(["rank_score"], ascending=False)
        .drop_duplicates(subset=["accession"], keep="first")
        .reset_index(drop=True)
    )

    # Optional: preserve step ordering if we had steps_order
    if steps_order is not None and not steps_order.empty:
        top_by_step = top_by_step.merge(
            steps_order.assign(_order=range(1, len(steps_order)+1)),
            on=["pathway_tag","step_idx"], how="left"
        ).sort_values(["_order","rank_score"], ascending=[True, False]).drop(columns=["_order"]).reset_index(drop=True)

    # --- Save outputs ---
    ranked_csv    = finish_out / "sequences_ranked_by_step.csv"
    shortlist_csv = finish_out / "sequences_shortlist.csv"
    top_by_step.to_csv(ranked_csv, index=False)
    shortlist.to_csv(shortlist_csv, index=False)

    # --- Compose state ---
    new_state = {
        **state,
        "sequences_ranked_csv": str(ranked_csv),
        "sequences_shortlist_csv": str(shortlist_csv),
        "sequence_rank_method": "score_norm + bacteria + preferred_host + acc_quality - blacklist",
        "sequence_rank_counts": {
            "steps": int(top_by_step[["pathway_tag","step_idx"]].drop_duplicates().shape[0]),
            "unique_accessions": int(shortlist.shape[0]),
            "rows_emitted": int(top_by_step.shape[0]),
            "top_n_per_step": TOP_N,
        }
    }
    new_state["logs"] = state.get("logs", []) + [
        f"sequence_rank: ranked {top_by_step.shape[0]} rows across {new_state['sequence_rank_counts']['steps']} steps",
        f"sequence_rank: unique shortlisted accessions = {shortlist.shape[0]}",
        f"sequence_rank: outputs={ranked_csv}, {shortlist_csv}"
    ]
    return new_state


    import os, pandas as pd
from pathlib import Path
from urllib.parse import quote_plus

def doe_node(state: "DBTLState") -> "DBTLState":
    """
    Build a Design-of-Experiments (DoE) screening sheet + markdown brief
    for the top-ranked pathway, and (if available) auto-fill per-step
    sequence fields from sequence_rank_node output.

    Inputs (from state):
      - ranked_csv or ranked_final_csv
      - steps_plan_csv or steps_annotated_csv
      - sequences_ranked_csv  (optional; produced by sequence_rank_node)

    Outputs:
      - {workdir}/retro_finish_out/enzyme_screening_sheet.csv
      - {workdir}/retro_finish_out/pathway_brief.md
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # --- Inputs ---
    rank_csv  = state.get("ranked_csv") or state.get("ranked_final_csv")
    steps_csv = (
        state.get("steps_plan_csv")
        or state.get("steps_annotated_csv")
        or str(finish_out / "steps_enzyme_plan.csv")
    )
    if not (rank_csv and Path(rank_csv).exists()):
        raise FileNotFoundError("doe_node: ranked_csv missing or not found.")
    if not (steps_csv and Path(steps_csv).exists()):
        raise FileNotFoundError("doe_node: steps CSV missing or not found.")

    ranked = pd.read_csv(rank_csv)
    steps  = pd.read_csv(steps_csv)
    ranked.columns = [c.strip() for c in ranked.columns]
    steps.columns  = [c.strip() for c in steps.columns]

    # --- Pick top pathway ---
    best_tag = str(ranked.iloc[0]["pathway_tag"])
    psteps = (
        steps[steps["pathway_tag"] == best_tag]
        .sort_values("step_idx")
        .reset_index(drop=True)
    )

    # --- Optional: bring in top-ranked sequence per step ---
    seq_csv = state.get("sequences_ranked_csv")
    merged_n = 0
    top_seq_by_step = None
    if seq_csv and Path(seq_csv).exists():
        seq = pd.read_csv(seq_csv)
        seq.columns = [c.strip() for c in seq.columns]
        # filter to our best pathway; pick top-1 per step by rank_score
        if "pathway_tag" in seq.columns and "rank_score" in seq.columns:
            seq_best = seq[seq["pathway_tag"] == best_tag].copy()
            if not seq_best.empty:
                seq_best["rank_score"] = pd.to_numeric(seq_best["rank_score"], errors="coerce").fillna(0.0)
                seq_best = (
                    seq_best.sort_values(["step_idx","rank_score"], ascending=[True, False])
                            .groupby("step_idx", as_index=False)
                            .first()
                )
                # keep tidy columns
                keep_cols = {
                    "step_idx":"step_idx",
                    "accession":"sequence_accession",
                    "organism":"source_organism",
                    "uniprot_url":"uniprot_url"
                }
                for k in keep_cols.keys():
                    if k not in seq_best.columns:
                        seq_best[k] = ""
                top_seq_by_step = seq_best[list(keep_cols.keys())].rename(columns=keep_cols)

    # --- Build screening sheet skeleton ---
    cols = [
        "pathway_tag","step_idx","retrobiocat_reaction","reaction_smiles",
        "selected_enzyme","possible_enzymes","selenzyme_url",
        "sequence_accession","source_organism","uniprot_url",
        "assay_buffer","pH","temp_C","cofactor","cofactor_recycle",
        "substrate_conc_mM","enzyme_loading_mg_per_mL","time_h",
        "conversion_%","notes"
    ]
    rows = []
    for _, r in psteps.iterrows():
        query = quote_plus(str(r.get("selected_enzyme") or r.get("retrobiocat_reaction")))
        uniprot_search = f"https://www.uniprot.org/uniprotkb?query={query}"
        rows.append({
            "pathway_tag": best_tag,
            "step_idx": int(r.get("step_idx", 0)),
            "retrobiocat_reaction": r.get("retrobiocat_reaction", ""),
            "reaction_smiles": r.get("reaction_smiles", ""),
            "selected_enzyme": r.get("selected_enzyme", ""),
            "possible_enzymes": r.get("possible_enzymes", ""),
            "selenzyme_url": r.get("selenzyme_url", ""),
            # placeholders (may be overwritten by sequence merge)
            "sequence_accession": "",
            "source_organism": "",
            "uniprot_url": uniprot_search,
            # DoE fields for lab to fill
            "assay_buffer": "",
            "pH": "",
            "temp_C": "",
            "cofactor": "",
            "cofactor_recycle": "",
            "substrate_conc_mM": "",
            "enzyme_loading_mg_per_mL": "",
            "time_h": "",
            "conversion_%": "",
            "notes": "",
        })
    df_screen = pd.DataFrame(rows, columns=cols)

    # Merge top sequence per step (if available)
    if top_seq_by_step is not None and not top_seq_by_step.empty:
        before_na = df_screen["sequence_accession"].isna().sum() + (df_screen["sequence_accession"] == "").sum()
        df_screen = df_screen.merge(top_seq_by_step, on="step_idx", how="left", suffixes=("","_best"))
        # prefer explicit accession from ranking; otherwise keep existing
        for col in ["sequence_accession","source_organism","uniprot_url"]:
            best_col = f"{col}_best"
            if best_col in df_screen.columns:
                df_screen[col] = df_screen[best_col].where(df_screen[best_col].notna() & (df_screen[best_col]!=""), df_screen[col])
                df_screen.drop(columns=[best_col], inplace=True)
        after_na = df_screen["sequence_accession"].isna().sum() + (df_screen["sequence_accession"] == "").sum()
        merged_n = max(0, before_na - after_na)

    # Write CSV
    screen_csv = str(finish_out / "enzyme_screening_sheet.csv")
    df_screen.to_csv(screen_csv, index=False)

    # --- Markdown brief ---
    md_path = str(finish_out / "pathway_brief.md")
    lines = []
    lines.append(f"# Pathway {best_tag} — Enzyme Screening Summary\n")
    lines.append(f"**Total Steps:** {len(psteps)}\n")
    lines.append(f"**Ranking File:** `{Path(rank_csv).name}`\n")

    def bullet(txt): return f"- {txt}"

    for _, r in psteps.iterrows():
        lines.append(f"\n## Step {int(r['step_idx'])}: {r.get('retrobiocat_reaction','')}\n")
        lines.append(bullet(f"Reaction SMILES: `{r.get('reaction_smiles','')}`"))
        lines.append(bullet(f"Selected enzyme: {r.get('selected_enzyme','-')}"))
        lines.append(bullet(f"Possible enzymes: {r.get('possible_enzymes','-')}"))
        if "precedent_best_similarity" in psteps.columns and pd.notna(r.get("precedent_best_similarity")):
            lines.append(bullet(f"Precedent similarity: {r.get('precedent_best_similarity')}"))
        if r.get("selenzyme_url"):
            lines.append(bullet(f"[Selenzyme link]({r.get('selenzyme_url')})"))
        # If we merged a top sequence for this step, surface it
        if merged_n and "sequence_accession" in df_screen.columns:
            acc = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"sequence_accession"].values[0]
            org = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"source_organism"].values[0]
            up  = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"uniprot_url"].values[0]
            if acc:
                lines.append(bullet(f"Top sequence: [{acc}]({up}) ({org})"))
        else:
            # fallback: include UniProt search for the selected enzyme name
            if r.get("selected_enzyme"):
                q = quote_plus(r.get("selected_enzyme"))
                lines.append(bullet(f"[UniProt search](https://www.uniprot.org/uniprotkb?query={q})"))

    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    # --- Update state/logs ---
    new_state = {
        **state,
        "screening_sheet_csv": screen_csv,
        "pathway_brief_md": md_path,
        "doe_pathway_tag": best_tag,
        "doe_steps": len(psteps),
        "doe_sequences_merged": int(merged_n),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"DoE sheet + brief ready for {best_tag}",
        f"screening={screen_csv}",
        f"brief={md_path}",
        f"sequences merged into DoE: {merged_n}",
    ]
    return new_state


from pathlib import Path
import pandas as pd, json

def human_gate_node(state: "DBTLState") -> "DBTLState":
    """
    Pause-and-review checkpoint node:
      - dumps a compact summary of key artifacts
      - waits for external 'approved' signal (LangGraph Interrupt equivalent)
      - by default sets approved=False
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    summary = {
        "top_pathway": state.get("doe_pathway_tag"),
        "steps": state.get("doe_steps"),
        "thermo_method": state.get("thermo_method"),
        "sequence_rank_counts": state.get("sequence_rank_counts", {}),
        "screening_sheet": state.get("screening_sheet_csv"),
        "brief": state.get("pathway_brief_md"),
    }

    gate_path = finish_out / "_HUMAN_GATE.md"
    with open(gate_path, "w", encoding="utf-8") as f:
        f.write("# Human Approval Gate\n\n")
        f.write("Please review the current design-build-test-learn outputs:\n\n")
        for k, v in summary.items():
            f.write(f"- **{k}**: {v}\n")
        f.write("\nMark `approved=True` in the next LangGraph signal to continue.\n")

    # detect external signal (if your LangGraph agents push state["signals"])
    signals = state.get("signals", {})
    approved = bool(signals.get("human_approved")) or state.get("approved", False)

    new_state = {
        **state,
        "approved": approved,
        "human_gate_md": str(gate_path),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"human_gate: approval file ready at {gate_path}",
        f"human_gate: approved={approved}",
    ]
    return new_state

import os, json, shutil, datetime
from pathlib import Path

def export_node(state: "DBTLState") -> "DBTLState":
    """
    Export all key DBTL artifacts (CSVs, MDs) and write a manifest.json.
    Optionally could push to ELN/LIMS/GSheet later.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    export_dir = workdir / "retro_export"
    export_dir.mkdir(parents=True, exist_ok=True)

    # Collect all relevant artifacts from state
    artifacts = []
    for k, v in state.items():
        if isinstance(v, str) and any(v.endswith(ext) for ext in [".csv",".md",".json",".xlsx"]):
            if os.path.exists(v):
                dst = export_dir / Path(v).name
                try:
                    shutil.copy2(v, dst)
                    artifacts.append(str(dst))
                except Exception as e:
                    state.setdefault("logs", []).append(f"export_node: failed to copy {v}: {e}")

    # Write manifest
    manifest = {
        "timestamp": datetime.datetime.now().isoformat(),
        "export_dir": str(export_dir),
        "artifacts": artifacts,
        "meta": {
            "pathway_tag": state.get("doe_pathway_tag"),
            "approved": state.get("approved"),
            "thermo_method": state.get("thermo_method"),
        },
    }
    manifest_path = export_dir / "manifest.json"
    with open(manifest_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)

    new_state = {
        **state,
        "export_manifest": str(manifest_path),
        "export_dir": str(export_dir),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"export_node: exported {len(artifacts)} artifacts",
        f"export_node: manifest={manifest_path}",
    ]
    return new_state

from langchain_core.runnables import RunnableLambda

import time
def wrap_node(name, fn):
    def _wrapped(state):
        t0 = time.time()
        state = {**state, "status": f"running:{name}", "last_node": name,
                 "logs": [*state.get("logs", []), f"▶ start {name}"]}
        out = fn(state)
        dt = time.time() - t0
        out_logs = [*out.get("logs", []), f"✔ done {name} ({dt:.1f}s)"]
        return {**out, "status": f"idle:{name}", "logs": out_logs, "last_node": name}
    return RunnableLambda(_wrapped)

def human_gate_node(state):
    approved = bool(state.get("signals", {}).get("human_approved")) or state.get("approved", False)
    logs = [*state.get("logs", [])]
    logs.append(f"human_gate: approved={approved}")
    return {**state, "approved": approved, "logs": logs}

def build_node(state: DBTLState) -> DBTLState:
    """
    Build phase: fetch enzyme sequences for each selected enzyme in steps_enzyme_plan.csv.
    - Queries UniProt/Selenzyme to fetch FASTA or metadata.
    - Writes sequences_build_plan.csv and FASTA bundle.
    """
    import os, pandas as pd, requests
    from urllib.parse import quote_plus
    from pathlib import Path

    steps_csv = Path(state["steps_plan_csv"])
    out_dir   = Path(state["workdir"]) / "build_out"
    out_dir.mkdir(parents=True, exist_ok=True)

    steps = pd.read_csv(steps_csv)
    seq_rows = []
    for _, r in steps.iterrows():
        enzyme = str(r.get("selected_enzyme", "")).strip()
        if not enzyme or enzyme == "nan":
            continue
        q = quote_plus(enzyme)
        uniprot_url = f"https://rest.uniprot.org/uniprotkb/search?query={q}&format=tsv&fields=accession,organism,protein_name,length"
        try:
            res = requests.get(uniprot_url, timeout=10)
            if res.ok:
                lines = res.text.splitlines()
                if len(lines) > 1:
                    acc, org, name, length = lines[1].split("\t")
                    seq_rows.append({
                        "step_idx": int(r["step_idx"]),
                        "selected_enzyme": enzyme,
                        "sequence_accession": acc,
                        "source_organism": org,
                        "uniprot_url": f"https://www.uniprot.org/uniprotkb/{acc}",
                        "protein_name": name,
                        "length": length,
                    })
        except Exception as e:
            seq_rows.append({"step_idx": r["step_idx"], "selected_enzyme": enzyme, "error": str(e)})

    seq_df = pd.DataFrame(seq_rows)
    seq_csv = out_dir / "sequence_candidates.csv"
    seq_df.to_csv(seq_csv, index=False)

    new_state = {
        **state,
        "build_out_dir": str(out_dir),
        "sequence_csv": str(seq_csv),
        **log(state, f"build: found {len(seq_rows)} sequence candidates")
    }
    return new_state


graph = StateGraph(DBTLState)

graph.add_node("retrosynthesis", wrap_node("retrosynthesis", retrosynthesis_node))
graph.add_node("extract",       wrap_node("extract",       extract_pathways_node))
graph.add_node("thermo",        wrap_node("thermo",        thermo_score_node))
graph.add_node("rank",          wrap_node("rank",          rank_node))
graph.add_node("selenzyme",     wrap_node("selenzyme",     selenzyme_node))
graph.add_node("doe",           wrap_node("doe",           doe_node))
graph.add_node("build", RunnableLambda(build_node))
graph.add_node("approve",       wrap_node("approve",       human_gate_node))
graph.add_node("export",        wrap_node("export",        export_node))

graph.set_entry_point("retrosynthesis")
graph.add_edge("retrosynthesis","extract")
graph.add_edge("extract","thermo")
graph.add_edge("thermo","rank")
graph.add_edge("rank","selenzyme")
graph.add_edge("selenzyme","doe")
graph.add_edge("doe", "build")
graph.add_edge("build", "approve")   # or -> simulate_node, if you prefer
graph.add_edge("doe","approve")

from langgraph.graph import END

graph.add_conditional_edges(
    "approve",
    lambda s: "export" if s.get("approved") else "__end__",   # ← return literal "__end__"
    {
        "export": "export",
        "__end__": END,                                       # ← map "__end__" to END
    },
)

graph.add_edge("export", END)

app = graph.compile()


In [5]:
# @title
init_state = {
  "run_id": "run_001",
  "target_smiles": "COC1=C(C=CC(=C1)C=O)O",
  "host": "Escherichia coli",
  "constraints": {"max_steps": "5"},
  "workdir": f"/content/runs/run_001",
  "logs": []
}

for state in app.stream(init_state, stream_mode="values"):
    node   = state.get("last_node", "?")
    status = state.get("status", "?")
    logs   = state.get("logs", [])
    last_log = logs[-1] if logs else ""     # ✅ safe guard
    print(f"[{node}] {status} | {last_log}")


[?] ? | 
[?] ? | ✔ done retrosynthesis (40.9s)
[?] ? | ✔ done extract (0.0s)
[?] ? | ✔ done thermo (0.6s)
[?] ? | ✔ done rank (0.0s)
[?] ? | ✔ done selenzyme (34.1s)
[?] ? | ✔ done doe (0.0s)


InvalidUpdateError: At key 'run_id': Can receive only one value per step. Use an Annotated key to handle multiple values.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_CONCURRENT_GRAPH_UPDATE

In [ ]:
New Version

In [22]:
# If needed in a fresh notebook:
!pip -q install langgraph langchain_core

from __future__ import annotations

import os, sys, re, json, time, math, glob, shutil, textwrap, datetime, subprocess
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Any
from typing_extensions import Annotated, TypedDict

import pandas as pd
from langgraph.graph import END, StateGraph
from langchain_core.runnables import RunnableLambda

# =========================
# State schema (fixes LastValue error by marking "run_id" static)
# =========================
class DBTLState(TypedDict, total=False):
    # Static keys (won’t change across steps; avoids InvalidUpdateError)
    run_id: Annotated[str, "static"]
    workdir: Annotated[str, "static"]

    # Inputs
    target_smiles: str
    host: str
    constraints: Dict[str, Any]

    # Bookkeeping
    status: str
    last_node: str
    logs: List[str]
    error: str
    signals: Dict[str, Any]
    approved: bool

    # Retro out
    retro_out_dir: str
    pathways_all_csv: Optional[str]
    pathways_solved_csv: Optional[str]
    pathways_all_json: Optional[str]
    pathways_solved_json: Optional[str]
    pathway_json_map: Dict[str, str]

    # Extractor / Thermo / Rank
    steps_plan_csv: Optional[str]
    steps_annotated_csv: Optional[str]
    ranked_csv: Optional[str]
    ranked_final_csv: Optional[str]
    ranking_method: Optional[str]
    thermo_method: Optional[str]

    # Selenzyme + sequences
    selenzyme_rows: int
    scraped_sequences_csv: Optional[str]
    sequences_ranked_csv: Optional[str]
    sequences_shortlist_csv: Optional[str]
    sequence_rank_method: Optional[str]
    sequence_rank_counts: Dict[str, int]

    # DoE
    screening_sheet_csv: Optional[str]
    pathway_brief_md: Optional[str]
    doe_pathway_tag: Optional[str]
    doe_steps: Optional[int]
    doe_sequences_merged: Optional[int]

    # Build
    build_out_dir: Optional[str]
    sequence_csv: Optional[str]

    # Export
    export_manifest: Optional[str]
    export_dir: Optional[str]
    human_gate_md: Optional[str]


def log(state: DBTLState, msg: str) -> Dict[str, List[str]]:
    return {"logs": (state.get("logs", []) + [msg])}


# ============== Helpers ==============
def _run(cmd, timeout=1800, env=None, cwd=None, check=True) -> subprocess.CompletedProcess:
    quiet = os.environ.get("DBTL_QUIET") == "1"
    # also let the graph control verbosity via constraints
    try:
        # best-effort: many nodes pass the same env; we check a global here too
        quiet = quiet or globals().get("_DBTL_QUIET", False)
    except Exception:
        pass

    if not quiet:
        print(f"[run] {cmd}")
    res = subprocess.run(
        cmd,
        shell=True,
        cwd=cwd,
        env=env or os.environ.copy(),
        stdout=(subprocess.DEVNULL if quiet else subprocess.PIPE),
        stderr=(subprocess.STDOUT if not quiet else subprocess.DEVNULL),
        text=True,
        timeout=timeout,
        check=check,
    )
    if not quiet and res.stdout:
        print(res.stdout)
    return res



def wrap_node(name: str, fn):
    """Add start/finish logs + timing to any node."""
    def _wrapped(state: DBTLState) -> DBTLState:
        t0 = time.time()
        pre = {**state,
               "status": f"running:{name}",
               "last_node": name,
               "logs": [*state.get("logs", []), f"▶ start {name}"]}
        out = fn(pre)
        dt = time.time() - t0
        out_logs = [*out.get("logs", []), f"✔ done {name} ({dt:.1f}s)"]
        return {**out, "status": f"idle:{name}", "logs": out_logs, "last_node": name}
    return RunnableLambda(_wrapped)


# ============== Nodes ==============

def retrosynthesis_node(state: DBTLState) -> DBTLState:
    """
    End-to-end RetroBioCat-2 retrosynthesis:
      - micromamba bootstrap (local to workdir)
      - create env + install RBC2 and pins
      - run MCTS on target_smiles
      - write all/solved pathways (CSV+JSON) + per-pathway JSONs
    """
    import shutil as _sh
    from pathlib import Path as _P

    try:
        workdir = _P(state["workdir"]).expanduser().resolve()
        target  = state.get("target_smiles")
        if not target:
            raise ValueError("retrosynthesis_node: state['target_smiles'] is required")

        cons = (state.get("constraints") or {})
        max_time   = int(cons.get("max_search_time", 15))
        expanders  = list(cons.get("expanders", ["retrobiocat"]))
        max_depth  = cons.get("max_depth", None)  # optional
        force_rerun= bool(cons.get("rerun", False))

        run_dir   = workdir
        retro_out = run_dir / "retro_out"
        retro_out.mkdir(parents=True, exist_ok=True)

        # micromamba layout inside run_dir
        mm_root  = run_dir / "micromamba"
        mm_bin   = run_dir / "bin" / "micromamba"
        env_pref = mm_root / "envs" / "retrobiocat"

        # bootstrap micromamba
        if not mm_bin.exists():
            _run(f"curl -Ls https://micro.mamba.pm/api/micromamba/linux-64/latest -o {run_dir}/mm.tar.bz2")
            _run(f"tar -xvjf {run_dir}/mm.tar.bz2 -C {run_dir} bin/micromamba")

        env = os.environ.copy()
        env["MAMBA_ROOT_PREFIX"] = str(mm_root)

        # create env if missing
        if force_rerun and env_pref.exists():
            _sh.rmtree(env_pref, ignore_errors=True)
        if not env_pref.exists():
            _run(f"{mm_bin} create -y -p {env_pref} -c conda-forge python=3.10 pip", env=env)

        # install RBC2 + pins
        cmds = [
            f"{mm_bin} run -p {env_pref} python -m pip install --upgrade pip setuptools wheel",
            # try zip first, then fallback to git
            f"{mm_bin} run -p {env_pref} python -m pip install https://github.com/willfinnigan/RetroBioCat-2/archive/refs/heads/main.zip"
            f" || {mm_bin} run -p {env_pref} python -m pip install 'git+https://github.com/willfinnigan/RetroBioCat-2.git'",
            f"{mm_bin} run -p {env_pref} python -m pip install 'pydantic<2' 'networkx<3' tqdm requests",
        ]
        for c in cmds:
            _run(c, env=env)

        # inline Python payload to run MCTS
        py = textwrap.dedent(f"""
            import json, pandas as pd
            from pathlib import Path
            from rbc2 import MCTS, get_expanders

            retro_out = Path(r"{retro_out}")
            retro_out.mkdir(parents=True, exist_ok=True)

            target_smi = r"{target}"
            expanders  = get_expanders({json.dumps(expanders)})

            mcts = MCTS(target_smi, expanders)
            mcts.config.max_search_time = int({max_time})
            {"mcts.config.max_depth = int(%d)" % int(max_depth) if max_depth is not None else ""}

            print("[rbc2] Running MCTS …")
            mcts.run()

            all_paths    = mcts.get_all_pathways()
            solved_paths = mcts.get_solved_pathways()

            def pathway_rows(pwy, tag):
                rows = []
                for i, rxn in enumerate(pwy.reactions, start=1):
                    rows.append({{
                        "pathway_tag": tag,
                        "step_idx": i,
                        "reaction_smiles": rxn.reaction_smiles(),
                        "product": rxn.product,
                        "substrates": " . ".join(rxn.substrates),
                        "rxn_type": rxn.rxn_type,
                        "name": rxn.name,
                        "score": rxn.score,
                    }})
                return rows

            def write_bundle(paths, prefix):
                all_rows = []
                tag_to_file = {{}}
                for k, pwy in enumerate(paths, start=1):
                    tag = f"P{{k:03d}}"
                    pjson = retro_out / f"{{tag}}.json"
                    with open(pjson, "w", encoding="utf-8") as f:
                        json.dump(pwy.save(), f, indent=2)
                    tag_to_file[tag] = str(pjson)
                    all_rows.extend(pathway_rows(pwy, tag))
                if all_rows:
                    df = pd.DataFrame(all_rows)
                    df.to_csv(retro_out / f"pathways_{{prefix}}_steps.csv", index=False)
                    with open(retro_out / f"pathways_{{prefix}}_steps.json", "w", encoding="utf-8") as f:
                        json.dump(all_rows, f, indent=2)
                return tag_to_file

            tag_map_all = write_bundle(all_paths, "all")
            tag_map_sol = write_bundle(solved_paths, "solved")

            print(f"[rbc2] Total pathways: {{len(all_paths)}} | Solved: {{len(solved_paths)}}")
            print("[rbc2] Outputs written to:", retro_out)
        """).strip()

        _run(f"""{mm_bin} run -p {env_pref} python - <<'PY'\n{py}\nPY""", env=env)

        # collect outputs
        pathways_all_csv      = str(retro_out / "pathways_all_steps.csv")
        pathways_solved_csv   = str(retro_out / "pathways_solved_steps.csv")
        pathways_all_json     = str(retro_out / "pathways_all_steps.json")
        pathways_solved_json  = str(retro_out / "pathways_solved_steps.json")
        pjson_files = glob.glob(str(retro_out / "P*.json"))
        tag_map = {Path(p).stem: str(Path(p)) for p in pjson_files}

        new_state = {
            **state,
            "retro_out_dir": str(retro_out),
            "pathways_all_csv": pathways_all_csv if os.path.exists(pathways_all_csv) else None,
            "pathways_solved_csv": pathways_solved_csv if os.path.exists(pathways_solved_csv) else None,
            "pathways_all_json": pathways_all_json if os.path.exists(pathways_all_json) else None,
            "pathways_solved_json": pathways_solved_json if os.path.exists(pathways_solved_json) else None,
            "pathway_json_map": tag_map,
        }
        new_state["logs"] = [*new_state.get("logs", []), "retrosynthesis done"]
        return new_state

    except subprocess.CalledProcessError as e:
        err = f"retrosynthesis_node: subprocess failed (code={e.returncode})"
        new_state = {**state, "error": err}
        new_state["logs"] = [*new_state.get("logs", []), err, str(e)]
        return new_state
    except Exception as e:
        err = f"retrosynthesis_node: {type(e).__name__}: {e}"
        new_state = {**state, "error": err}
        new_state["logs"] = [*new_state.get("logs", []), err]
        return new_state


def extract_pathways_node(state: DBTLState) -> DBTLState:
    """
    Build enzyme-planning artifacts from RetroBioCat-2 outputs:
      - steps_enzyme_plan.csv (per-step details incl. enzyme classes, precedents, Selenzyme links)
      - pathways_ranked_no_thermo.csv (RBC2 + precedent-based ranking; no thermodynamics)
    """
    workdir = Path(state["workdir"])
    retro_out = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # pick CSV (solved preferred, else all)
    csv_solved = retro_out / "pathways_solved_steps.csv"
    csv_all    = retro_out / "pathways_all_steps.csv"
    csv_in = csv_solved if csv_solved.exists() else csv_all
    if not csv_in.exists():
        raise FileNotFoundError(f"No pathways CSV found under {retro_out}")

    steps_df = pd.read_csv(csv_in)

    # pathway -> json map
    json_map: Dict[str,str] = dict(state.get("pathway_json_map") or {})
    if not json_map:
        for p in glob.glob(str(retro_out / "P*.json")):
            tag = Path(p).stem  # P001
            json_map[tag] = p

    from urllib.parse import quote_plus

    def selenzyme_url_from_rxn(rxn_smiles: str, org: str) -> str:
        return (
            "https://selenzyme.synbiochem.co.uk/selenzy/selenzy?reaction_smiles="
            + quote_plus(rxn_smiles)
            + "&organism="
            + quote_plus(org)
        )

    host_org = state.get("host") or "Escherichia coli"

    # expand per-step details from Pxxx.json
    records = []
    for _, r in steps_df.iterrows():
        tag   = str(r.get("pathway_tag"))
        idx   = int(r.get("step_idx"))
        rxn   = str(r.get("reaction_smiles",""))
        prod  = str(r.get("products", r.get("product","")))
        subs  = str(r.get("substrates",""))

        rxn_type = r.get("rxn_type","")
        name     = r.get("name","")
        rbc2_score = r.get("score", float("nan"))

        possible_enzymes = []
        enzyme_choices = []
        selected_enzyme = ""
        precedents = []

        pj = json_map.get(tag)
        if pj and os.path.exists(pj):
            try:
                data = json.load(open(pj, "r"))
                if isinstance(data, list) and 1 <= idx <= len(data):
                    d = data[idx - 1]
                    rxn_type = d.get("rxn_type", rxn_type)
                    name     = d.get("name", name)
                    tm = d.get("template_metadata", {}) or {}
                    if tm:
                        tkey = next(iter(tm.keys()))
                        tmeta = tm.get(tkey, {}) or {}
                        possible_enzymes = (tmeta.get("possible_enzymes") or [])
                        choices = (tmeta.get("enzyme_choices") or [])
                        enzyme_choices = [
                            " / ".join(x) if isinstance(x, (list, tuple)) else str(x)
                            for x in choices
                        ]
                        selected_enzyme = tmeta.get("selected_enzyme") or ""
                    precedents = d.get("precedents") or []
            except Exception:
                pass

        prec_sorted = sorted(precedents, key=lambda x: x.get("similarity", 0), reverse=True)[:3]
        prec_summ, best_sim = [], float("nan")
        for p in prec_sorted:
            sim = p.get("similarity", float("nan"))
            dat = p.get("data", {}) or {}
            doi = dat.get("html_doi", "") or dat.get("doi", "")
            enz = dat.get("enzyme_name", "") or p.get("name", "")
            cite = dat.get("short_citation", "")
            try:
                prec_summ.append(f"{enz} | sim={float(sim):.3f} | {cite} | {doi}")
            except Exception:
                prec_summ.append(f"{enz} | sim={sim} | {cite} | {doi}")
        if prec_sorted:
            best_sim = prec_sorted[0].get("similarity", float("nan"))

        records.append({
            "pathway_tag": tag,
            "step_idx": idx,
            "rxn_type": rxn_type,
            "retrobiocat_reaction": name,
            "reaction_smiles": rxn,
            "substrates": subs,
            "products": prod,
            "rbc2_score": rbc2_score,
            "selected_enzyme": selected_enzyme,
            "possible_enzymes": " ; ".join(possible_enzymes),
            "enzyme_choices": " ; ".join(enzyme_choices),
            "precedent_top3": " || ".join(prec_summ),
            "precedent_best_similarity": best_sim,
            "selenzyme_url": selenzyme_url_from_rxn(rxn, host_org),
        })

    plan = pd.DataFrame(records).sort_values(["pathway_tag","step_idx"]).reset_index(drop=True)

    # pathway ranking (no thermo)
    tmp = plan.copy()
    tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
    tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)

    agg = tmp.groupby("pathway_tag").agg(
        steps_count=("step_idx","max"),
        sum_rbc2=("rbc2_score","sum"),
        sum_prec_sim=("precedent_best_similarity","sum"),
    ).reset_index()
    agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
    ranked = agg.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False, False, False, True]
    ).reset_index(drop=True)

    steps_plan_csv = str(finish_out / "steps_enzyme_plan.csv")
    ranked_csv     = str(finish_out / "pathways_ranked_no_thermo.csv")
    plan.to_csv(steps_plan_csv, index=False)
    ranked.to_csv(ranked_csv, index=False)

    new_state = {
        **state,
        "steps_plan_csv": steps_plan_csv,
        "ranked_csv": ranked_csv,
        "thermo_method": "none",
    }
    new_state["logs"] = state.get("logs", []) + [
        f"extracted pathways ({len(plan)} step rows, {len(ranked)} pathways)",
        f"steps_plan={steps_plan_csv}",
        f"ranked_no_thermo={ranked_csv}",
    ]
    return new_state


def thermo_score_node(state: DBTLState) -> DBTLState:
    """
    Lightweight 'match-only' thermo pass (no ChemAxon / ΔG):
      - Emits steps_annotated.csv and pathways_ranked.csv (same heuristic as extractor).
    """
    workdir    = Path(state["workdir"])
    mm_root    = workdir / "micromamba"
    env_prefix = mm_root / "envs" / "retrobiocat"  # same env
    retro_out  = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    steps_plan_csv = state.get("steps_plan_csv")
    if not steps_plan_csv or not Path(steps_plan_csv).exists():
        csv_solved = retro_out / "pathways_solved_steps.csv"
        csv_all    = retro_out / "pathways_all_steps.csv"
        steps_plan_csv = str(csv_solved if csv_solved.exists() else csv_all)

    steps_out = str(finish_out / "steps_annotated.csv")
    rank_out  = str(finish_out / "pathways_ranked.csv")

    py = f"""\
import pandas as pd
from pathlib import Path

IN  = Path(r"{steps_plan_csv}")
OUT_STEPS = Path(r"{steps_out}")
OUT_RANK  = Path(r"{rank_out}")

if not IN.exists():
    raise FileNotFoundError(f"Input steps table not found: {{IN}}")

df = pd.read_csv(IN)

# Ensure required columns exist
for col in ["pathway_tag","step_idx","reaction_smiles","substrates","products",
            "rbc2_score","precedent_best_similarity","selenzyme_url"]:
    if col not in df.columns:
        df[col] = "" if col not in ["rbc2_score","precedent_best_similarity"] else 0.0

# Add placeholders for thermo
df["dGprime_kJ_per_mol"] = float("nan")
df["uncert_kJ_per_mol"]  = float("nan")
df["thermo_pass"]        = False
df["equilibrator_formula"] = ""

# Save annotated
df.to_csv(OUT_STEPS, index=False)

# Ranking (same as extractor)
tmp = df.copy()
tmp["rbc2_score"] = pd.to_numeric(tmp["rbc2_score"], errors="coerce").fillna(0.0)
tmp["precedent_best_similarity"] = pd.to_numeric(tmp["precedent_best_similarity"], errors="coerce").fillna(0.0)

agg = tmp.groupby("pathway_tag").agg(
    steps_count=("step_idx","max"),
    sum_rbc2=("rbc2_score","sum"),
    sum_prec_sim=("precedent_best_similarity","sum"),
).reset_index()

agg["rank_score"] = agg["sum_rbc2"] + 0.2*agg["sum_prec_sim"] - 0.1*agg["steps_count"]
ranked = agg.sort_values(["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
                         ascending=[False,False,False,True]).reset_index(drop=True)
ranked.to_csv(OUT_RANK, index=False)

print("Saved:")
print(" ", OUT_STEPS)
print(" ", OUT_RANK)
"""
    micromamba = workdir / "bin" / "micromamba"
    if not micromamba.exists():
        micromamba = "micromamba"

    env = os.environ.copy()
    env["MAMBA_ROOT_PREFIX"] = str(mm_root)

    try:
        _run(f"""{micromamba} run -p "{env_prefix}" python - <<'PY'\n{py}\nPY""", env=env)
    except subprocess.CalledProcessError:
        _run(f"python - <<'PY'\n{py}\nPY")

    coverage = 0
    method = "matchonly"

    new_state = {
        **state,
        "thermo_method": method,
        "steps_annotated_csv": steps_out,
        "ranked_csv": rank_out,
    }
    new_state["logs"] = state.get("logs", []) + [
        f"thermo: {method}, coverage={coverage}",
        f"steps_annotated={steps_out}",
        f"ranked={rank_out}",
    ]
    return new_state


def rank_node(state: DBTLState) -> DBTLState:
    """
    Merge thermo info (if available) into pathway ranking; else keep RBC2+precedent ranking.
    Emits pathways_ranked_final.csv
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not steps_csv or not Path(steps_csv).exists():
        raise FileNotFoundError("rank_node: steps CSV not found in state")

    df = pd.read_csv(steps_csv)
    df.columns = [c.strip() for c in df.columns]

    has_thermo = any(c in df.columns for c in ["dGprime_kJ_per_mol", "thermo_pass"])
    thermo_mode = bool(has_thermo and df["thermo_pass"].any())  # likely False in match-only

    df["rbc2_score"] = pd.to_numeric(df.get("rbc2_score", 0.0), errors="coerce").fillna(0.0)
    df["precedent_best_similarity"] = pd.to_numeric(df.get("precedent_best_similarity", 0.0),
                                                    errors="coerce").fillna(0.0)
    if "dGprime_kJ_per_mol" in df:
        df["dGprime_kJ_per_mol"] = pd.to_numeric(df["dGprime_kJ_per_mol"], errors="coerce")

    group = df.groupby("pathway_tag", dropna=True)

    if thermo_mode:
        ranked = (
            group.agg(
                steps_count=("step_idx","max"),
                sum_rbc2=("rbc2_score","sum"),
                sum_prec_sim=("precedent_best_similarity","sum"),
                thermo_pass_steps=("thermo_pass","sum"),
                sum_dGprime=("dGprime_kJ_per_mol","sum"),
            )
            .reset_index()
        )
        ranked["all_steps_pass"] = ranked["thermo_pass_steps"] == ranked["steps_count"]
        ranked["rank_score"] = (
            ranked["sum_rbc2"]
            + 0.2 * ranked["sum_prec_sim"]
            - 0.1 * ranked["steps_count"]
            - 0.001 * ranked["sum_dGprime"].fillna(0.0)
            + ranked["all_steps_pass"].astype(float) * 0.5
        )
        method = "rbc2+precedent+thermo"
    else:
        ranked = (
            group.agg(
                steps_count=("step_idx","max"),
                sum_rbc2=("rbc2_score","sum"),
                sum_prec_sim=("precedent_best_similarity","sum"),
            )
            .reset_index()
        )
        ranked["rank_score"] = (
            ranked["sum_rbc2"] + 0.2 * ranked["sum_prec_sim"] - 0.1 * ranked["steps_count"]
        )
        method = "rbc2+precedent"

    ranked = ranked.sort_values(
        ["rank_score","sum_rbc2","sum_prec_sim","steps_count"],
        ascending=[False, False, False, True],
    ).reset_index(drop=True)

    out_csv = str(finish_out / "pathways_ranked_final.csv")
    ranked.to_csv(out_csv, index=False)

    new_state = {
        **state,
        "ranked_final_csv": out_csv,
        "ranking_method": method,
    }
    new_state["logs"] = state.get("logs", []) + [
        f"ranked pathways ({method}, {len(ranked)} entries)",
        f"ranked_final={out_csv}",
    ]
    return new_state


def selenzyme_node(state: DBTLState) -> DBTLState:
    """
    Collect Selenzyme links and try to scrape top sequences (polite, tolerant).
    Falls back to counting links if blocked. Emits selenzyme_scrape.csv if rows found.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not steps_csv or not Path(steps_csv).exists():
        raise FileNotFoundError("selenzyme_node: no steps CSV found in state")

    steps = pd.read_csv(steps_csv)
    if "selenzyme_url" not in steps.columns:
        new_state = {
            **state,
            "selenzyme_rows": 0,
            "scraped_sequences_csv": None,
        }
        new_state["logs"] = state.get("logs", []) + ["selenzyme: no 'selenzyme_url' column; skipped"]
        return new_state

    url_map: Dict[str, List[Tuple[str,int]]] = {}
    for _, r in steps.iterrows():
        url = str(r.get("selenzyme_url") or "").strip()
        if not url:
            continue
        tag = str(r.get("pathway_tag"))
        idx = int(r.get("step_idx", 0))
        url_map.setdefault(url, []).append((tag, idx))

    unique_urls = list(url_map.keys())

    cfg = (state.get("constraints", {}) or {}).get("selenzyme", {}) or {}
    do_scrape: bool   = bool(cfg.get("scrape", True))
    max_urls: int     = int(cfg.get("max_urls", 40))
    timeout_s: int    = int(cfg.get("timeout_s", 15))
    sleep_s: float    = float(cfg.get("sleep_s", 1.0))

    urls_to_hit = unique_urls[:max_urls]
    scraped_rows: List[Dict] = []
    blocked_reason: Optional[str] = None

    def _try_scrape(url: str) -> List[Dict]:
        nonlocal blocked_reason
        try:
            import requests
            from bs4 import BeautifulSoup  # type: ignore

            headers = {
                "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0 Safari/537.36",
                "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
                "Connection": "keep-alive",
            }
            resp = requests.get(url, headers=headers, timeout=timeout_s)
            if resp.status_code in (403, 429):
                raise RuntimeError(f"HTTP {resp.status_code}")
            if not resp.ok or not resp.text:
                return []
            html = resp.text
            soup = BeautifulSoup(html, "html.parser")

            table = None
            for t in soup.find_all("table"):
                ths = [th.get_text(strip=True).lower() for th in t.find_all("th")]
                joined = " ".join(ths)
                if any(k in joined for k in ["uniprot", "organism", "score", "ec", "enzyme"]):
                    table = t
                    break

            rows_local: List[Dict] = []
            if table:
                headers = [th.get_text(strip=True) for th in table.find_all("th")]
                idx_map = {h.lower(): i for i, h in enumerate(headers)}

                def _get(td_list, key_variants):
                    for k in key_variants:
                        i = idx_map.get(k)
                        if i is not None and i < len(td_list):
                            return td_list[i].get_text(strip=True)
                    return ""

                for tr in table.find_all("tr"):
                    tds = tr.find_all("td")
                    if not tds:
                        continue
                    acc = _get(tds, ["uniprot", "accession", "uniprot id"])
                    org = _get(tds, ["organism", "source", "species"])
                    scr = _get(tds, ["score", "similarity", "sim"])
                    ec  = _get(tds, ["ec", "ec number"])
                    enz = _get(tds, ["enzyme", "name", "protein"])

                    if not acc:
                        a = tds[0].find("a")
                        if a and a.get_text(strip=True):
                            acc = a.get_text(strip=True)
                    if not acc:
                        m = re.search(r"[A-NR-Z0-9]{{6,10}}", tr.get_text(" ", strip=True))
                        if m:
                            acc = m.group(0)

                    try:
                        scr_val = float(re.sub("[^0-9.+-eE]", "", scr)) if scr else None
                    except Exception:
                        scr_val = None

                    rows_local.append({
                        "accession": acc,
                        "organism": org,
                        "score": scr_val if scr_val is not None else scr,
                        "ec": ec,
                        "enzyme_name": enz,
                    })

            if not rows_local:
                tokens = re.findall(r"\b[OPQ][0-9][A-Z0-9]{{3}}[0-9]\b|\b[A-NR-Z0-9]{{6}}\b", html)
                rows_local = [{"accession": t, "organism": "", "score": "", "ec": "", "enzyme_name": ""} for t in dict.fromkeys(tokens)]

            return rows_local

        except Exception as e:
            blocked_reason = str(e)
            return []

    if do_scrape and len(urls_to_hit) > 0:
        for i, url in enumerate(urls_to_hit, 1):
            rows_local = _try_scrape(url)
            for (tag, idx) in url_map.get(url, []):
                for r in rows_local:
                    scraped_rows.append({
                        "pathway_tag": tag,
                        "step_idx": idx,
                        "selenzyme_url": url,
                        **r
                    })
            if i < len(urls_to_hit):
                try:
                    time.sleep(sleep_s)
                except Exception:
                    pass

    scraped_csv_path: Optional[str] = None
    if scraped_rows:
        out_csv = finish_out / "selenzyme_scrape.csv"
        pd.DataFrame(scraped_rows, columns=[
            "pathway_tag","step_idx","selenzyme_url",
            "accession","organism","score","ec","enzyme_name"
        ]).to_csv(out_csv, index=False)
        scraped_csv_path = str(out_csv)

    total_links = len(unique_urls)
    rows = len(scraped_rows)

    log_lines = [
        f"selenzyme: links={total_links}, scraped_rows={rows}, mode={'scrape' if do_scrape else 'count-only'}"
    ]
    if blocked_reason and rows == 0 and do_scrape:
        log_lines.append(f"selenzyme: scrape blocked or failed ({blocked_reason}); fell back to count-only")

    new_state = {
        **state,
        "selenzyme_rows": rows,
        "scraped_sequences_csv": scraped_csv_path,
    }
    new_state["logs"] = state.get("logs", []) + log_lines
    return new_state


def sequence_rank_node(state: DBTLState) -> DBTLState:
    """
    Rank Selenzyme-scraped sequences per step and emit:
      - sequences_ranked_by_step.csv
      - sequences_shortlist.csv
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    scraped_csv = state.get("scraped_sequences_csv")
    if not scraped_csv or not Path(scraped_csv).exists():
        new_state = { **state }
        new_state["logs"] = state.get("logs", []) + ["sequence_rank: no scraped_sequences_csv; skipped"]
        return new_state

    steps_csv = state.get("steps_plan_csv")
    steps_order = None
    if steps_csv and Path(steps_csv).exists():
        try:
            _steps = pd.read_csv(steps_csv)
            steps_order = (
                _steps[["pathway_tag","step_idx"]]
                .drop_duplicates()
                .sort_values(["pathway_tag","step_idx"])
            )
        except Exception:
            steps_order = None

    cfg: Dict = (state.get("constraints", {}) or {}).get("sequence_rank", {}) or {}
    TOP_N = int(cfg.get("top_n_per_step", 5))
    preferred_hosts = set(cfg.get("preferred_hosts", []))
    extra_bact = set(cfg.get("extra_bacterial_genera", ["Pseudomonas","Corynebacterium","Rhodococcus","Acinetobacter"]))
    blacklist_re = re.compile(cfg.get("blacklist_regex", r"(mitochondrial|chloroplast)"), flags=re.I)

    df = pd.read_csv(scraped_csv)
    for col in ["pathway_tag","step_idx","selenzyme_url","accession","organism","score","ec","enzyme_name"]:
        if col not in df.columns:
            df[col] = "" if col not in ["score","step_idx"] else (0.0 if col=="score" else 0)

    df["step_idx"] = pd.to_numeric(df["step_idx"], errors="coerce").fillna(0).astype(int)
    def _to_float(x):
        try:
            if pd.isna(x): return math.nan
            return float(str(x).strip())
        except Exception:
            return math.nan
    df["score_num"] = df["score"].apply(_to_float)

    bacterial_keywords = [
        "bacter", "bacillus","escherichia","pseudomonas","corynebacterium","rhodococcus",
        "acinetobacter","enterobacter","salmonella","shigella","staphylococcus","streptococcus",
        "lactobacillus","lactococcus","klebsiella","burkholderia","clostridium","vibrio",
        "shewanella","xanthomonas","mycobacterium","yersinia","pasteurella","azotobacter",
        "cupriavidus","caulobacter","alcaligenes","sphingomonas"
    ] + [g.lower() for g in extra_bact]
    euk_exclude = [
        "homo","mus","rattus","danio","drosophila","bos","gallus","arabidopsis","oryza","zea",
        "nicotiana","saccharomyces","pichia","kluyveromyces","aspergillus","neurospora","candida"
    ]
    def is_bacterial(org: str) -> bool:
        org_l = (org or "").lower()
        if any(k in org_l for k in euk_exclude):
            return False
        if any(k in org_l for k in bacterial_keywords):
            return True
        return False
    df["is_bacterial"] = df["organism"].apply(is_bacterial)

    def in_preferred(org: str) -> bool:
        if not preferred_hosts: return False
        for ph in preferred_hosts:
            if ph.lower() in (org or "").lower():
                return True
        return False
    df["is_preferred_host"] = df["organism"].apply(in_preferred)
    df["is_blacklisted"] = df["organism"].apply(lambda x: bool(blacklist_re.search(str(x))) if str(x) else False)

    def acc_quality(acc: str) -> int:
        s = (acc or "").strip()
        return 1 if re.fullmatch(r"[A-NR-Z0-9]{6,10}", s) else 0
    df["acc_quality"] = df["accession"].apply(acc_quality)

    s = df["score_num"]
    if s.notna().sum() > 0:
        def per_step_norm(g):
            vals = g["score_num"]
            vmin = vals.min(skipna=True)
            vmax = vals.max(skipna=True)
            if pd.isna(vmin) or pd.isna(vmax) or vmin == vmax:
                return pd.Series([0.5]*len(g), index=g.index)
            return (vals - vmin) / (vmax - vmin)
        df["score_norm"] = df.groupby(["pathway_tag","step_idx"], dropna=False, as_index=False).apply(per_step_norm).reset_index(level=[0,1], drop=True)
        df["score_norm"] = df["score_norm"].fillna(0.5)
    else:
        df["score_norm"] = 0.5

    df["bonus_bacterial"] = df["is_bacterial"].astype(float) * 0.20
    df["bonus_preferred"] = df["is_preferred_host"].astype(float) * 0.10
    df["bonus_acc"]       = df["acc_quality"].astype(float) * 0.05
    df["pen_blacklist"]   = df["is_blacklisted"].astype(float) * 0.30

    df["rank_score"] = df["score_norm"] + df["bonus_bacterial"] + df["bonus_preferred"] + df["bonus_acc"] - df["pen_blacklist"]

    df = df.sort_values(["accession","rank_score"], ascending=[True, False])
    df_dedup = df.drop_duplicates(subset=["accession"], keep="first")

    top_by_step = (
        df_dedup
        .sort_values(["pathway_tag","step_idx","rank_score"], ascending=[True, True, False])
        .groupby(["pathway_tag","step_idx"], dropna=False)
        .head(int(TOP_N))
        .reset_index(drop=True)
    )

    def uniprot_url(acc: str) -> str:
        acc = str(acc).strip()
        return f"https://www.uniprot.org/uniprotkb/{acc}" if acc else ""
    top_by_step["uniprot_url"] = top_by_step["accession"].apply(uniprot_url)

    cols = [
        "pathway_tag","step_idx","accession","organism","ec","enzyme_name",
        "score","score_num","score_norm",
        "is_bacterial","is_preferred_host","is_blacklisted","acc_quality",
        "bonus_bacterial","bonus_preferred","bonus_acc","pen_blacklist",
        "rank_score","selenzyme_url","uniprot_url"
    ]
    cols = [c for c in cols if c in top_by_step.columns]
    top_by_step = top_by_step[cols]

    shortlist = (
        top_by_step
        .sort_values(["rank_score"], ascending=False)
        .drop_duplicates(subset=["accession"], keep="first")
        .reset_index(drop=True)
    )

    if steps_order is not None and not steps_order.empty:
        top_by_step = top_by_step.merge(
            steps_order.assign(_order=range(1, len(steps_order)+1)),
            on=["pathway_tag","step_idx"], how="left"
        ).sort_values(["_order","rank_score"], ascending=[True, False]).drop(columns=["_order"]).reset_index(drop=True)

    ranked_csv    = finish_out / "sequences_ranked_by_step.csv"
    shortlist_csv = finish_out / "sequences_shortlist.csv"
    top_by_step.to_csv(ranked_csv, index=False)
    shortlist.to_csv(shortlist_csv, index=False)

    new_state = {
        **state,
        "sequences_ranked_csv": str(ranked_csv),
        "sequences_shortlist_csv": str(shortlist_csv),
        "sequence_rank_method": "score_norm + bacteria + preferred_host + acc_quality - blacklist",
        "sequence_rank_counts": {
            "steps": int(top_by_step[["pathway_tag","step_idx"]].drop_duplicates().shape[0]),
            "unique_accessions": int(shortlist.shape[0]),
            "rows_emitted": int(top_by_step.shape[0]),
            "top_n_per_step": int(TOP_N),
        }
    }
    new_state["logs"] = state.get("logs", []) + [
        f"sequence_rank: ranked {top_by_step.shape[0]} rows across {new_state['sequence_rank_counts']['steps']} steps",
        f"sequence_rank: unique shortlisted accessions = {shortlist.shape[0]}",
        f"sequence_rank: outputs={ranked_csv}, {shortlist_csv}"
    ]
    return new_state


def doe_node(state: DBTLState) -> DBTLState:
    """
    Build a DoE screening sheet + markdown brief for the top-ranked pathway.
    Also merges the top-ranked sequence per step (if available).
    """
    from urllib.parse import quote_plus

    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    rank_csv  = state.get("ranked_csv") or state.get("ranked_final_csv")
    steps_csv = state.get("steps_plan_csv") or state.get("steps_annotated_csv") or str(finish_out / "steps_enzyme_plan.csv")
    if not (rank_csv and Path(rank_csv).exists()):
        raise FileNotFoundError("doe_node: ranked_csv missing or not found.")
    if not (steps_csv and Path(steps_csv).exists()):
        raise FileNotFoundError("doe_node: steps CSV missing or not found.")

    ranked = pd.read_csv(rank_csv)
    steps  = pd.read_csv(steps_csv)
    ranked.columns = [c.strip() for c in ranked.columns]
    steps.columns  = [c.strip() for c in steps.columns]

    best_tag = str(ranked.iloc[0]["pathway_tag"])
    psteps = (
        steps[steps["pathway_tag"] == best_tag]
        .sort_values("step_idx")
        .reset_index(drop=True)
    )

    # bring in top-ranked sequence per step (optional)
    seq_csv = state.get("sequences_ranked_csv")
    merged_n = 0
    top_seq_by_step = None
    if seq_csv and Path(seq_csv).exists():
        seq = pd.read_csv(seq_csv)
        seq.columns = [c.strip() for c in seq.columns]
        if "pathway_tag" in seq.columns and "rank_score" in seq.columns:
            seq_best = seq[seq["pathway_tag"] == best_tag].copy()
            if not seq_best.empty:
                seq_best["rank_score"] = pd.to_numeric(seq_best["rank_score"], errors="coerce").fillna(0.0)
                seq_best = (
                    seq_best.sort_values(["step_idx","rank_score"], ascending=[True, False])
                            .groupby("step_idx", as_index=False)
                            .first()
                )
                keep_cols = {
                    "step_idx":"step_idx",
                    "accession":"sequence_accession",
                    "organism":"source_organism",
                    "uniprot_url":"uniprot_url"
                }
                for k in keep_cols.keys():
                    if k not in seq_best.columns:
                        seq_best[k] = ""
                top_seq_by_step = seq_best[list(keep_cols.keys())].rename(columns=keep_cols)

    cols = [
        "pathway_tag","step_idx","retrobiocat_reaction","reaction_smiles",
        "selected_enzyme","possible_enzymes","selenzyme_url",
        "sequence_accession","source_organism","uniprot_url",
        "assay_buffer","pH","temp_C","cofactor","cofactor_recycle",
        "substrate_conc_mM","enzyme_loading_mg_per_mL","time_h",
        "conversion_%","notes"
    ]
    rows = []
    for _, r in psteps.iterrows():
        query = quote_plus(str(r.get("selected_enzyme") or r.get("retrobiocat_reaction")))
        uniprot_search = f"https://www.uniprot.org/uniprotkb?query={query}"
        rows.append({
            "pathway_tag": best_tag,
            "step_idx": int(r.get("step_idx", 0)),
            "retrobiocat_reaction": r.get("retrobiocat_reaction", ""),
            "reaction_smiles": r.get("reaction_smiles", ""),
            "selected_enzyme": r.get("selected_enzyme", ""),
            "possible_enzymes": r.get("possible_enzymes", ""),
            "selenzyme_url": r.get("selenzyme_url", ""),
            "sequence_accession": "",
            "source_organism": "",
            "uniprot_url": uniprot_search,
            "assay_buffer": "",
            "pH": "",
            "temp_C": "",
            "cofactor": "",
            "cofactor_recycle": "",
            "substrate_conc_mM": "",
            "enzyme_loading_mg_per_mL": "",
            "time_h": "",
            "conversion_%": "",
            "notes": "",
        })
    df_screen = pd.DataFrame(rows, columns=cols)

    if top_seq_by_step is not None and not top_seq_by_step.empty:
        before_na = df_screen["sequence_accession"].isna().sum() + (df_screen["sequence_accession"] == "").sum()
        df_screen = df_screen.merge(top_seq_by_step, on="step_idx", how="left", suffixes=("","_best"))
        for col in ["sequence_accession","source_organism","uniprot_url"]:
            best_col = f"{col}_best"
            if best_col in df_screen.columns:
                df_screen[col] = df_screen[best_col].where(df_screen[best_col].notna() & (df_screen[best_col]!=""), df_screen[col])
                df_screen.drop(columns=[best_col], inplace=True)
        after_na = df_screen["sequence_accession"].isna().sum() + (df_screen["sequence_accession"] == "").sum()
        merged_n = max(0, before_na - after_na)

    screen_csv = str(finish_out / "enzyme_screening_sheet.csv")
    df_screen.to_csv(screen_csv, index=False)

    md_path = str(finish_out / "pathway_brief.md")
    lines = []
    lines.append(f"# Pathway {best_tag} — Enzyme Screening Summary\n")
    lines.append(f"**Total Steps:** {len(psteps)}\n")
    lines.append(f"**Ranking File:** `{Path(rank_csv).name}`\n")

    def bullet(txt): return f"- {txt}"

    for _, r in psteps.iterrows():
        lines.append(f"\n## Step {int(r['step_idx'])}: {r.get('retrobiocat_reaction','')}\n")
        lines.append(bullet(f"Reaction SMILES: `{r.get('reaction_smiles','')}`"))
        lines.append(bullet(f"Selected enzyme: {r.get('selected_enzyme','-')}"))
        lines.append(bullet(f"Possible enzymes: {r.get('possible_enzymes','-')}"))
        if "precedent_best_similarity" in psteps.columns and pd.notna(r.get("precedent_best_similarity")):
            lines.append(bullet(f"Precedent similarity: {r.get('precedent_best_similarity')}"))
        if r.get("selenzyme_url"):
            lines.append(bullet(f"[Selenzyme link]({r.get('selenzyme_url')})"))
        if merged_n and "sequence_accession" in df_screen.columns:
            acc = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"sequence_accession"].values[0]
            org = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"source_organism"].values[0]
            up  = df_screen.loc[df_screen["step_idx"]==int(r["step_idx"]),"uniprot_url"].values[0]
            if acc:
                lines.append(bullet(f"Top sequence: [{acc}]({up}) ({org})"))
        else:
            if r.get("selected_enzyme"):
                q = quote_plus(r.get("selected_enzyme"))
                lines.append(bullet(f"[UniProt search](https://www.uniprot.org/uniprotkb?query={q})"))

    with open(md_path, "w", encoding="utf-8") as f:
        f.write("\n".join(lines))

    new_state = {
        **state,
        "screening_sheet_csv": screen_csv,
        "pathway_brief_md": md_path,
        "doe_pathway_tag": best_tag,
        "doe_steps": len(psteps),
        "doe_sequences_merged": int(merged_n),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"DoE sheet + brief ready for {best_tag}",
        f"screening={screen_csv}",
        f"brief={md_path}",
        f"sequences merged into DoE: {merged_n}",
    ]
    return new_state


def human_gate_node(state: DBTLState) -> DBTLState:
    """
    Pause-and-review checkpoint node. Writes a small summary file and
    checks for an external 'signals.human_approved' flag.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    summary = {
        "top_pathway": state.get("doe_pathway_tag"),
        "steps": state.get("doe_steps"),
        "thermo_method": state.get("thermo_method"),
        "sequence_rank_counts": state.get("sequence_rank_counts", {}),
        "screening_sheet": state.get("screening_sheet_csv"),
        "brief": state.get("pathway_brief_md"),
    }

    gate_path = finish_out / "_HUMAN_GATE.md"
    with open(gate_path, "w", encoding="utf-8") as f:
        f.write("# Human Approval Gate\n\n")
        f.write("Please review the current design-build-test-learn outputs:\n\n")
        for k, v in summary.items():
            f.write(f"- **{k}**: {v}\n")
        f.write("\nMark `approved=True` in the next LangGraph signal to continue.\n")

    signals = state.get("signals", {})
    approved = bool(signals.get("human_approved")) or state.get("approved", False)

    new_state = {
        **state,
        "approved": approved,
        "human_gate_md": str(gate_path),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"human_gate: approval file ready at {gate_path}",
        f"human_gate: approved={approved}",
    ]
    return new_state


def export_node(state: DBTLState) -> DBTLState:
    """
    Export all key DBTL artifacts and write a manifest.json.
    """
    workdir = Path(state["workdir"])
    finish_out = workdir / "retro_finish_out"
    export_dir = workdir / "retro_export"
    export_dir.mkdir(parents=True, exist_ok=True)

    artifacts = []
    for k, v in state.items():
        if isinstance(v, str) and any(v.endswith(ext) for ext in [".csv",".md",".json",".xlsx"]):
            if os.path.exists(v):
                dst = export_dir / Path(v).name
                try:
                    shutil.copy2(v, dst)
                    artifacts.append(str(dst))
                except Exception as e:
                    state.setdefault("logs", []).append(f"export_node: failed to copy {v}: {e}")

    manifest = {
        "timestamp": datetime.datetime.now().isoformat(),
        "export_dir": str(export_dir),
        "artifacts": artifacts,
        "meta": {
            "pathway_tag": state.get("doe_pathway_tag"),
            "approved": state.get("approved"),
            "thermo_method": state.get("thermo_method"),
        },
    }
    manifest_path = export_dir / "manifest.json"
    with open(manifest_path, "w", encoding="utf-8") as f:
        json.dump(manifest, f, indent=2)

    new_state = {
        **state,
        "export_manifest": str(manifest_path),
        "export_dir": str(export_dir),
    }
    new_state["logs"] = state.get("logs", []) + [
        f"export_node: exported {len(artifacts)} artifacts",
        f"export_node: manifest={manifest_path}",
    ]
    return new_state


def build_node(state: DBTLState) -> DBTLState:
    """
    (Optional) BUILD phase stub:
    Query UniProt for quick candidates from selected enzyme names.
    Writes build_out/sequence_candidates.csv
    """
    try:
        import requests
    except Exception:
        # If no internet or requests missing, just noop gracefully.
        new_state = {**state, **log(state, "build: requests not available or offline; skipped") }
        return new_state

    from urllib.parse import quote_plus

    steps_csv = state.get("steps_plan_csv")
    if not steps_csv or not os.path.exists(steps_csv):
        return {**state, **log(state, "build: no steps_plan_csv; skipped")}

    out_dir   = Path(state["workdir"]) / "build_out"
    out_dir.mkdir(parents=True, exist_ok=True)

    steps = pd.read_csv(steps_csv)
    seq_rows = []
    for _, r in steps.iterrows():
        enzyme = str(r.get("selected_enzyme", "")).strip()
        if not enzyme or enzyme == "nan":
            continue
        q = quote_plus(enzyme)
        uniprot_url = f"https://rest.uniprot.org/uniprotkb/search?query={q}&format=tsv&fields=accession,organism,protein_name,length"
        try:
            res = requests.get(uniprot_url, timeout=10)
            if res.ok:
                lines = res.text.splitlines()
                if len(lines) > 1:
                    parts = lines[1].split("\t")
                    if len(parts) >= 4:
                        acc, org, name, length = parts[0], parts[1], parts[2], parts[3]
                        seq_rows.append({
                            "step_idx": int(r["step_idx"]),
                            "selected_enzyme": enzyme,
                            "sequence_accession": acc,
                            "source_organism": org,
                            "uniprot_url": f"https://www.uniprot.org/uniprotkb/{acc}",
                            "protein_name": name,
                            "length": length,
                        })
        except Exception as e:
            seq_rows.append({"step_idx": r["step_idx"], "selected_enzyme": enzyme, "error": str(e)})

    seq_df = pd.DataFrame(seq_rows)
    seq_csv = out_dir / "sequence_candidates.csv"
    seq_df.to_csv(seq_csv, index=False)

    new_state = {
        **state,
        "build_out_dir": str(out_dir),
        "sequence_csv": str(seq_csv),
        **log(state, f"build: found {len(seq_rows)} sequence candidates")
    }
    return new_state

def simulate_node(state: "DBTLState") -> "DBTLState":
    """
    Simulate pathway feasibility (toy COBRApy flux demo + mass-balance check).
    Writes:
      - retro_finish_out/simulation_fluxes.csv
      - retro_finish_out/simulation_log.txt
    Falls back gracefully if COBRApy is unavailable.
    """
    import os, subprocess, textwrap, json, shutil, time
    from pathlib import Path
    from string import Template

    def _log(s, msg):
        s = {**s}
        s["logs"] = [*s.get("logs", []), msg]
        return s

    workdir = Path(state["workdir"]).expanduser().resolve()
    retro_out = workdir / "retro_out"
    finish_out = workdir / "retro_finish_out"
    finish_out.mkdir(parents=True, exist_ok=True)

    # Inputs
    rank_csv  = state.get("ranked_final_csv") or state.get("ranked_csv")
    steps_csv = state.get("steps_annotated_csv") or state.get("steps_plan_csv")
    if not rank_csv or not Path(rank_csv).exists():
        return _log(state, "simulate: no ranking CSV; skipped")
    if not steps_csv or not Path(steps_csv).exists():
        return _log(state, "simulate: no steps CSV; skipped")

    # Outputs
    out_flux = finish_out / "simulation_fluxes.csv"
    log_path = finish_out / "simulation_log.txt"

    # Use the same micromamba env as RBC2
    mm_root    = workdir / "micromamba"
    env_prefix = mm_root / "envs" / "retrobiocat"
    micromamba = workdir / "bin" / "micromamba"
    if not micromamba.exists():
        micromamba = "micromamba"

    env = os.environ.copy()
    env["MAMBA_ROOT_PREFIX"] = str(mm_root)

    # Ensure cobra is present (best-effort)
    try:
        subprocess.run(
            f"""{micromamba} run -p "{env_prefix}" python -c "import cobra" """,
            shell=True, env=env, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )
        cobra_ok = True
    except subprocess.CalledProcessError:
        cobra_ok = False
        try:
            subprocess.run(
                f"""{micromamba} run -p "{env_prefix}" python -m pip install cobra""",
                shell=True, env=env, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
            )
            cobra_ok = True
        except subprocess.CalledProcessError:
            cobra_ok = False

    # ---- Embedded script (Template with $placeholders to avoid {} issues) ----
    tpl = Template(r"""
import pandas as pd, math, csv, sys
from pathlib import Path

RANK_CSV = Path("$rank_csv")
STEPS_CSV = Path("$steps_csv")
OUT_FLUX = Path("$out_flux")
LOG_PATH = Path("$log_path")

log_lines = []

def log(x):
    log_lines.append(str(x))

ranked = pd.read_csv(RANK_CSV)
steps  = pd.read_csv(STEPS_CSV)
best_tag = str(ranked.iloc[0]["pathway_tag"])

psteps = (steps[steps["pathway_tag"] == best_tag]
          .sort_values("step_idx")
          .reset_index(drop=True))

# Basic mass-balance parse: count participants LHS/RHS for sanity
def split_rxn(r):
    if not isinstance(r, str) or ">>" not in r: return [], []
    L,R = r.split(">>",1)
    subs=[s for s in L.split(".") if s]; prods=[p for p in R.split(".") if p]
    return subs, prods

imbalances = []
for _, row in psteps.iterrows():
    subs, prods = split_rxn(str(row.get("reaction_smiles","")))
    imbalances.append(abs(len(subs) - len(prods)))
mass_ok = (sum(imbalances) == 0)
log(f"mass_balance_ok={mass_ok} (total_imbalance={sum(imbalances)})")

# If COBRApy is available, create a toy model and assign unit flux per step
try:
    import cobra
    model = cobra.Model("toy_pathway")
    # Make exchange for overall substrate mix and product pool
    exch_in  = cobra.Reaction("EX_subs")
    exch_out = cobra.Reaction("EX_prod")
    model.add_reactions([exch_in, exch_out])
    exch_in.lower_bound  = -1000.0; exch_in.upper_bound  = 0.0
    exch_out.lower_bound = 0.0;     exch_out.upper_bound = 1000.0

    # create a metabolite per unique token just to wire reactions
    tokens = set()
    for rxn in psteps["reaction_smiles"].astype(str):
        L,R = split_rxn(rxn)
        tokens.update(L); tokens.update(R)
    # Minimal placeholders; we don't use stoichiometries beyond 1
    mets = {}
    for t in tokens:
        m_id = ("m_" + "".join([c if c.isalnum() else "_" for c in t]))[:60]
        mets[t] = cobra.Metabolite(id=m_id, name=t, compartment="c")

    # Add step reactions (each consumes its LHS tokens and produces RHS tokens)
    step_flux = []
    for _, row in psteps.iterrows():
        sid = f"R_step_{int(row['step_idx'])}"
        rxn = cobra.Reaction(sid)
        L,R = split_rxn(str(row.get("reaction_smiles","")))
        rxn.lower_bound = 0.0; rxn.upper_bound = 1000.0
        rxn.add_metabolites({ mets[s]: -1.0 for s in L if s in mets })
        rxn.add_metabolites({ mets[p]:  1.0 for p in R if p in mets })
        model.add_reactions([rxn])
        step_flux.append((sid, 1.0))  # target steady-state flux guess

    # Objective: push flux through the last step
    if step_flux:
        last_sid = step_flux[-1][0]
        model.objective = last_sid

    sol = model.optimize()
    fluxes = []
    if sol.status == "optimal":
        for sid, _ in step_flux:
            fluxes.append({"pathway_tag": best_tag,
                           "step_id": sid,
                           "flux": float(sol.fluxes.get(sid, 0.0))})
        flux_ok = True
    else:
        flux_ok = False
        for sid, _ in step_flux:
            fluxes.append({"pathway_tag": best_tag, "step_id": sid, "flux": float("nan")})

    pd.DataFrame(fluxes).to_csv(OUT_FLUX, index=False)
    log(f"cobra_status={sol.status}, flux_ok={flux_ok}, rows={len(fluxes)}")

except Exception as e:
    # COBRA missing or failed — emit a stub CSV with zeros
    fluxes = []
    for _, row in psteps.iterrows():
        fluxes.append({"pathway_tag": best_tag,
                       "step_id": f"R_step_{int(row['step_idx'])}",
                       "flux": 0.0})
    pd.DataFrame(fluxes).to_csv(OUT_FLUX, index=False)
    log(f"cobra_unavailable_or_failed: {e}; wrote zero-flux stub")

with open(LOG_PATH, "w", encoding="utf-8") as f:
    f.write("\n".join(log_lines))
print("simulate: wrote", OUT_FLUX, "and", LOG_PATH)
""")

    py = textwrap.dedent(tpl.substitute(
        rank_csv=str(rank_csv),
        steps_csv=str(steps_csv),
        out_flux=str(out_flux),
        log_path=str(log_path),
    ))

    # run inside env
    try:
        subprocess.run(
            f"""{micromamba} run -p "{env_prefix}" python - <<'PY'\n{py}\nPY""",
            shell=True, env=env, check=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT
        )
    except subprocess.CalledProcessError as e:
        # if even that fails, record and continue
        with open(log_path, "a", encoding="utf-8") as f:
            f.write("\nsubprocess failed:\n" + (e.stdout or ""))

    new_state = {
        **state,
        "simulation_flux_csv": str(out_flux) if out_flux.exists() else None,
        "simulation_log": str(log_path) if log_path.exists() else None,
    }
    new_state["logs"] = [*state.get("logs", []), f"simulate: fluxes={out_flux.exists()}, log={log_path.exists()}"]
    return new_state



# =========================
# Wire the graph
# =========================
graph = StateGraph(DBTLState)

# Nodes (unchanged)
graph.add_node("retrosynthesis", wrap_node("retrosynthesis", retrosynthesis_node))
graph.add_node("extract",       wrap_node("extract",       extract_pathways_node))
graph.add_node("thermo",        wrap_node("thermo",        thermo_score_node))
graph.add_node("rank",          wrap_node("rank",          rank_node))
graph.add_node("simulate",      wrap_node("simulate",      simulate_node))
graph.add_node("selenzyme",     wrap_node("selenzyme",     selenzyme_node))
graph.add_node("seqrank",       wrap_node("seqrank",       sequence_rank_node))
graph.add_node("doe",           wrap_node("doe",           doe_node))
graph.add_node("build",         wrap_node("build",         build_node))
graph.add_node("approve",       wrap_node("approve",       human_gate_node))
graph.add_node("export",        wrap_node("export",        export_node))

# Edges: simulate runs AFTER rank and BEFORE selenzyme.
graph.set_entry_point("retrosynthesis")
graph.add_edge("retrosynthesis","extract")
graph.add_edge("extract","thermo")
graph.add_edge("thermo","rank")
graph.add_edge("rank","simulate")
graph.add_edge("simulate","selenzyme")
graph.add_edge("selenzyme","seqrank")
graph.add_edge("seqrank","doe")
graph.add_edge("doe","build")
graph.add_edge("build","approve")

# Approve: either proceed to export or end
graph.add_conditional_edges(
    "approve",
    lambda s: "export" if s.get("approved") else "__end__",
    {"export": "export", "__end__": END},
)
graph.add_edge("export", END)

app = graph.compile()


# =========================
# Example run
# =========================
# init_state = DBTLState(
#     run_id="run_001",
#    workdir="/content/runs/run_001",
#     target_smiles="COC1=C(C=CC(=C1)C=O)O",
#     host="Escherichia coli",
#     constraints={"max_steps":"5"},
#     logs=[]
# )
# for state in app.stream(init_state, stream_mode="values"):
#     node   = state.get("last_node", "?")
#     status = state.get("status", "?")
#     print(f"[{node}] {status}")
# print("Final logs:")
# print("\n".join(state.get("logs", [])))


In [23]:
import os
os.environ["DBTL_QUIET"] = "1"   # silence micromamba/pip/stdout


init_state = {
  "run_id": "run_001",
  "workdir": "/content/runs/run_001",
  "target_smiles": "COC1=C(C=CC(=C1)C=O)O",
  "host": "Escherichia coli",
  "constraints": {"max_steps": "5"},
  "signals": {"human_approved": True},   # ← approve
  "approved": True,
  "logs": []
}

for state in app.stream(init_state, stream_mode="values"):
    pass

print("Export manifest:", state.get("export_manifest"))
print("Export dir:", state.get("export_dir"))


Export manifest: /content/runs/run_001/retro_export/manifest.json
Export dir: /content/runs/run_001/retro_export
